In [2]:
# Import necessary libraries
from binance.client import Client
import os
from dotenv import load_dotenv # Import the function

# Load environment variables from .env file
load_dotenv() 

# Now access the environment variables
# These names MUST match the names used in your .env file
api_key = os.environ.get('BINANCE_API_KEY') 
api_secret = os.environ.get('BINANCE_API_SECRET')

# Verify if the keys were loaded successfully
print("API Key loaded:", api_key is not None)
print("API Secret loaded:", api_secret is not None) 

# Optional: Print a few characters of the API key to be sure (NEVER print the secret)
if api_key:
    print("API Key starts with:", api_key[:5]) 

API Key loaded: True
API Secret loaded: True
API Key starts with: GVnoN


In [3]:
# Make sure api_key and api_secret are loaded from the previous cell

# Initialize the Binance Client
# IMPORTANT: Use tld='us' for Binance.US
client = Client(api_key, api_secret, tld='us')

# Test the connection with a simple ping
try:
    ping_result = client.ping()
    print("Successfully pinged Binance.US API!")
    # ping_result should be an empty dictionary {} on success
    print("Ping result:", ping_result) 
except Exception as e:
    print(f"Error connecting to Binance.US API: {e}")

Successfully pinged Binance.US API!
Ping result: {}


In [4]:
# Fetch account information (includes balances)
try:
    account_info = client.get_account()
    
    # Print the keys available in the account_info dictionary
    # This helps understand the structure of the response
    print("Available keys in account info:", account_info.keys()) 
    
    # Extract and print balances (usually a list of dictionaries)
    balances = account_info.get('balances', []) 
    print("\n--- Account Balances ---")
    found_assets = False
    for asset in balances:
        # Only print assets with a non-zero balance (free or locked)
        if float(asset['free']) > 0 or float(asset['locked']) > 0:
            print(f"Asset: {asset['asset']}, Free: {asset['free']}, Locked: {asset['locked']}")
            found_assets = True
    
    if not found_assets:
        print("No assets with a balance found (or only zero balances).")
        
except Exception as e:
    print(f"Error fetching account information: {e}")

Available keys in account info: dict_keys(['makerCommission', 'takerCommission', 'buyerCommission', 'sellerCommission', 'commissionRates', 'canTrade', 'canWithdraw', 'canDeposit', 'brokered', 'requireSelfTradePrevention', 'updateTime', 'accountType', 'balances', 'permissions'])

--- Account Balances ---
Asset: BTC, Free: 0.00002710, Locked: 0.00040000
Asset: BUSD, Free: 0.14956400, Locked: 0.00000000
Asset: WAVES, Free: 0.01000000, Locked: 0.00000000


In [5]:
# Define the trading pair symbol
# Symbols on Binance typically combine the base and quote asset without slashes (e.g., BTCUSDT)
symbol = 'BTCUSDT' 

try:
    # Fetch the latest ticker price information for the symbol
    ticker_info = client.get_symbol_ticker(symbol=symbol)
    
    print(f"--- Ticker Information for {symbol} ---")
    print(f"Symbol: {ticker_info.get('symbol')}")
    print(f"Price: {ticker_info.get('price')}") 
    
    # You can print the whole dictionary to see all available info
    # print("\nFull Ticker Info:")
    # print(ticker_info)

except Exception as e:
    print(f"Error fetching ticker information for {symbol}: {e}")
    # Common error: Invalid symbol if the pair doesn't exist on Binance.US

--- Ticker Information for BTCUSDT ---
Symbol: BTCUSDT
Price: 78121.09000000


In [6]:
import pandas as pd

# Define symbol and interval
symbol = 'BTCUSDT'
# Options: KLINE_INTERVAL_1MINUTE, KLINE_INTERVAL_5MINUTE, KLINE_INTERVAL_1HOUR, KLINE_INTERVAL_1DAY, etc.
interval = Client.KLINE_INTERVAL_1HOUR 
limit = 100 # Number of candles to retrieve (max 1000)

try:
    print(f"Fetching last {limit} klines for {symbol} on interval {interval}...")
    # Fetch klines (candlestick data)
    # Returns a list of lists: [open_time, open, high, low, close, volume, close_time, ...]
    klines_raw = client.get_klines(symbol=symbol, interval=interval, limit=limit)

    # Define columns based on Binance API documentation for klines
    columns = [
        'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 
        'Close Time', 'Quote Asset Volume', 'Number of Trades', 
        'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
    ]
    
    # Create Pandas DataFrame
    klines_df = pd.DataFrame(klines_raw, columns=columns)

    # --- Data Cleaning and Formatting ---
    # Convert timestamp columns to datetime objects (timestamps are in milliseconds)
    klines_df['Open Time'] = pd.to_datetime(klines_df['Open Time'], unit='ms')
    klines_df['Close Time'] = pd.to_datetime(klines_df['Close Time'], unit='ms')

    # Convert relevant numeric columns to float/numeric type
    numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 
                    'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume']
    for col in numeric_cols:
        klines_df[col] = pd.to_numeric(klines_df[col])
        
    # Set 'Open Time' as the index (optional but common)
    # klines_df.set_index('Open Time', inplace=True)

    print(f"\n--- Last 5 Klines for {symbol} ({interval}) ---")
    # Display the last 5 rows of the DataFrame
    print(klines_df.tail()) 

    print(f"\nDataFrame shape: {klines_df.shape}") # Should show (limit, 12) e.g. (100, 12)

except Exception as e:
    print(f"Error fetching klines for {symbol}: {e}")

Fetching last 100 klines for BTCUSDT on interval 1h...

--- Last 5 Klines for BTCUSDT (1h) ---
             Open Time      Open      High       Low     Close   Volume  \
95 2025-04-07 13:00:00  77273.71  78420.96  75834.16  78420.96  4.02179   
96 2025-04-07 14:00:00  78160.53  81188.05  77965.44  79000.09  5.87123   
97 2025-04-07 15:00:00  79000.00  79337.51  77652.61  78790.54  1.79442   
98 2025-04-07 16:00:00  78496.71  78497.82  77362.16  77362.16  0.93518   
99 2025-04-07 17:00:00  77520.28  78366.54  77520.28  77787.58  0.08578   

                Close Time  Quote Asset Volume  Number of Trades  \
95 2025-04-07 13:59:59.999       309064.823581               442   
96 2025-04-07 14:59:59.999       465379.411499              1005   
97 2025-04-07 15:59:59.999       140580.766465               292   
98 2025-04-07 16:59:59.999        73191.965486               199   
99 2025-04-07 17:59:59.999         6684.676928                47   

    Taker Buy Base Asset Volume  Taker Buy Qu

In [7]:
import pandas as pd
import os
from datetime import datetime
from binance.client import Client # Make sure this library is installed
from dotenv import load_dotenv     # Make sure this library is installed

# --- Load API Keys from .env file ---
load_dotenv() 
api_key = os.environ.get('BINANCE_API_KEY') 
api_secret = os.environ.get('BINANCE_API_SECRET')

# --- Initialize Binance Client ---
# Check if keys are loaded before initializing
if not api_key or not api_secret:
    print("Error: API Key or Secret not found in environment variables.")
    print("Ensure you have a .env file with BINANCE_API_KEY and BINANCE_API_SECRET.")
    # You might want to raise an error or exit here in a real script
    client = None 
else:
    try:
        client = Client(api_key, api_secret, tld='us')
        # Test connection briefly
        client.ping()
        print("Binance client initialized successfully.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None

# --- Configuration for Data Fetching ---
symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_1HOUR # e.g., 1h candles
start_date_str = "1 Jan, 2023" # Example: Fetch data starting from beginning of 2023
data_directory = "data" # Folder to save the data

# --- Main Data Fetching Logic ---
if client: # Only proceed if the client was initialized successfully
    
    # --- Ensure data directory exists ---
    os.makedirs(data_directory, exist_ok=True)
    
    # --- Define the output filename ---
    # Create a filename safe for file systems
    start_date_safe = start_date_str.replace(' ','_').replace(',','').replace(':','-')
    filename = os.path.join(data_directory, f"{symbol}_{interval}_{start_date_safe}_to_now.csv")
    
    print(f"\nAttempting to fetch data for {symbol} interval {interval} since {start_date_str}")
    print(f"Data will be saved to: {filename}")

    try:
        # Fetch historical klines using the generator function for pagination
        klines_generator = client.get_historical_klines_generator(
            symbol, 
            interval, 
            start_date_str, 
            # end_str=None, # Defaults to now if not provided
            # limit=1000 # Default chunk size
        )

        # --- Process and Save ---
        # Convert generator to list (this might take time and memory for long periods)
        print("Fetching data from API (this may take some time)...")
        all_klines = list(klines_generator) 
        
        if not all_klines:
            print("No data found for the specified parameters.")
        else:
            print(f"Fetched {len(all_klines)} klines.")
            
            # Define columns based on Binance API documentation for klines
            columns = [
                'Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 
                'Close Time', 'Quote Asset Volume', 'Number of Trades', 
                'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
            ]
            
            # Create Pandas DataFrame
            klines_df = pd.DataFrame(all_klines, columns=columns)

            # --- Data Cleaning ---
            klines_df['Open Time'] = pd.to_datetime(klines_df['Open Time'], unit='ms')
            klines_df['Close Time'] = pd.to_datetime(klines_df['Close Time'], unit='ms')
            numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 
                            'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume']
            for col in numeric_cols:
                klines_df[col] = pd.to_numeric(klines_df[col])
            
            # Drop the 'Ignore' column as it's not useful
            klines_df.drop('Ignore', axis=1, inplace=True) 

            # --- Save to CSV ---
            # Set 'Open Time' as index before saving for easier time-based retrieval later
            klines_df.set_index('Open Time', inplace=True) 
            klines_df.to_csv(filename) 
            print(f"Data successfully saved to {filename}")
            
            # Display first and last few rows of the saved data
            print("\n--- First 5 rows of fetched data ---")
            print(klines_df.head())
            print("\n--- Last 5 rows of fetched data ---")
            print(f"\nDataFrame shape: {klines_df.shape}")

    except Exception as e:
        print(f"An error occurred during data fetching/processing: {e}")
else:
    print("\nSkipping data fetching because the Binance client failed to initialize.")

Binance client initialized successfully.

Attempting to fetch data for BTCUSDT interval 1h since 1 Jan, 2023
Data will be saved to: data/BTCUSDT_1h_1_Jan_2023_to_now.csv
Fetching data from API (this may take some time)...
Fetched 19859 klines.
Data successfully saved to data/BTCUSDT_1h_1_Jan_2023_to_now.csv

--- First 5 rows of fetched data ---
                         Open      High       Low     Close     Volume  \
Open Time                                                                
2023-01-01 00:00:00  16524.41  16533.00  16500.63  16526.09  16.810355   
2023-01-01 01:00:00  16528.29  16546.99  16518.79  16546.99  11.604272   
2023-01-01 02:00:00  16537.70  16554.60  16529.80  16539.38   9.440864   
2023-01-01 03:00:00  16536.57  16537.40  16508.76  16521.36  12.603051   
2023-01-01 04:00:00  16530.82  16533.60  16514.69  16520.57   6.139097   

                                 Close Time  Quote Asset Volume  \
Open Time                                                         


In [8]:
import pandas as pd
import os

# --- Configuration ---
# Make sure these match the parameters used when saving the file
symbol = 'BTCUSDT'
interval = '1h' # Use the string representation matching the saved filename if needed
start_date_str = "1 Jan, 2023" 
data_directory = "data" 

# Construct the filename EXACTLY as it was saved
start_date_safe = start_date_str.replace(' ','_').replace(',','').replace(':','-')
filename = os.path.join(data_directory, f"{symbol}_{interval}_{start_date_safe}_to_now.csv")

print(f"Attempting to load data from: {filename}")

try:
    # Check if the file exists before trying to load
    if os.path.exists(filename):
        # Load the CSV file using pandas
        # 'index_col=0' uses the first column (Open Time) as the index
        # 'parse_dates=True' attempts to automatically parse the index column as dates
        loaded_df = pd.read_csv(filename, index_col=0, parse_dates=True) 
        
        print("Data loaded successfully.")
        
        # Display first few rows to verify
        print("\n--- First 5 rows of loaded data ---")
        print(loaded_df.head())
        
        # Display DataFrame info to check data types
        print("\n--- DataFrame Info ---")
        loaded_df.info()
 "An error occurred while loading the data: {e}")

Attempting to load data from: data/BTCUSDT_1h_1_Jan_2023_to_now.csv
Data loaded successfully.

--- First 5 rows of loaded data ---
                         Open      High       Low     Close     Volume  \
Open Time                                                                
2023-01-01 00:00:00  16524.41  16533.00  16500.63  16526.09  16.810355   
2023-01-01 01:00:00  16528.29  16546.99  16518.79  16546.99  11.604272   
2023-01-01 02:00:00  16537.70  16554.60  16529.80  16539.38   9.440864   
2023-01-01 03:00:00  16536.57  16537.40  16508.76  16521.36  12.603051   
2023-01-01 04:00:00  16530.82  16533.60  16514.69  16520.57   6.139097   

                                  Close Time  Quote Asset Volume  \
Open Time                                                          
2023-01-01 00:00:00  2023-01-01 00:59:59.999       277705.479312   
2023-01-01 01:00:00  2023-01-01 01:59:59.999       191826.844309   
2023-01-01 02:00:00  2023-01-01 02:59:59.999       156143.896745   
2023-01-01

In [11]:
import pandas as pd
import os
from binance.client import Client # Make sure this library is installed
from dotenv import load_dotenv     # Make sure this library is installed

# --- Load API Keys from .env file ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

# --- Initialize Binance Client ---
client = None # Initialize client as None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping() # Quick check
        print("Binance client initialized successfully for balance check.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Function to Get Balances ---
def get_current_balances(api_client):
    """
    Fetches account balances from Binance.US and returns non-zero balances
    as a Pandas DataFrame.

    Args:
        api_client: An initialized Binance API Client object.

    Returns:
        pandas.DataFrame: DataFrame with columns ['Asset', 'Free', 'Locked'],
                          indexed by 'Asset', containing only non-zero balances.
                          Returns an empty DataFrame on error or if client is None.
    """
    if not api_client:
        print("API Client is not available.")
        return pd.DataFrame(columns=['Asset', 'Free', 'Locked']).set_index('Asset')

    try:
        account_info = api_client.get_account()
        balances_raw = account_info.get('balances', [])

        processed_balances = []
        for asset_info in balances_raw:
            free = float(asset_info['free'])
            locked = float(asset_info['locked'])
            # Only include assets where at least one balance is > 0
            if free > 0 or locked > 0:
                processed_balances.append({
                    'Asset': asset_info['asset'],
                    'Free': free,
                    'Locked': locked
                })

        if not processed_balances:
            print("No assets with non-zero balance found.")
            return pd.DataFrame(columns=['Asset', 'Free', 'Locked']).set_index('Asset')

        # Convert to DataFrame and set Asset as index for easy lookup
        balances_df = pd.DataFrame(processed_balances)
        balances_df.set_index('Asset', inplace=True)
        return balances_df

    except Exception as e:
        print(f"Error fetching account balances: {e}")
        return pd.DataFrame(columns=['Asset', 'Free', 'Locked']).set_index('Asset')

# --- Example Usage ---
if client: # Only call if client is valid
    current_balances_df = get_current_balances(client)
    print("\n--- Current Non-Zero Balances ---")
    if not current_balances_df.empty:
        print(current_balances_df)
    else:
        print("Could not retrieve balances or balances are empty.")

else:
    print("\nCannot fetch balances because client failed to initialize.")

# Note: The erroneous instruction line from the previous response has been removed.

Binance client initialized successfully for balance check.

--- Current Non-Zero Balances ---
           Free  Locked
Asset                  
BTC    0.000027  0.0004
BUSD   0.149564  0.0000
WAVES  0.010000  0.0000


In [13]:
import pandas as pd
import os
from binance.client import Client # Make sure this library is installed
from dotenv import load_dotenv     # Make sure this library is installed
import time # Import time for potential rate limiting delays

# --- Load API Keys and Initialize Client (assuming client is needed again) ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None # Initialize client as None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping() # Quick check
        print("Binance client initialized successfully for value calculation.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Function Definition (assuming get_current_balances exists or include it) ---
# Re-include the function if running this cell independently
def get_current_balances(api_client):
    if not api_client: return pd.DataFrame(columns=['Asset', 'Free', 'Locked']).set_index('Asset')
    try:
        account_info = api_client.get_account()
        balances_raw = account_info.get('balances', [])
        processed_balances = []
        for asset_info in balances_raw:
            free = float(asset_info['free'])
            locked = float(asset_info['locked'])
            if free > 0 or locked > 0:
                processed_balances.append({'Asset': asset_info['asset'], 'Free': free, 'Locked': locked})
        if not processed_balances: return pd.DataFrame(columns=['Asset', 'Free', 'Locked']).set_index('Asset')
        balances_df = pd.DataFrame(processed_balances)
        balances_df.set_index('Asset', inplace=True)
        return balances_df
    except Exception as e:
        print(f"Error fetching account balances: {e}")
        return pd.DataFrame(columns=['Asset', 'Free', 'Locked']).set_index('Asset')

# --- Configuration ---
quote_asset = 'USDT' # The asset to value the portfolio in (USDT, USDC, BUSD often used)

# --- Main Value Calculation Logic ---
if client:
    balances_df = get_current_balances(client)

    if balances_df.empty:
        print("Cannot calculate value: No balances found or error fetching balances.")
    else:
        print(f"\nCalculating portfolio value in {quote_asset}...")
        
        total_portfolio_value = 0.0
        asset_values = {} # Dictionary to store details

        for asset, row in balances_df.iterrows():
            quantity = row['Free'] + row['Locked'] # Total quantity of the asset
            current_price = 0.0
            usdt_value = 0.0

            if asset == quote_asset:
                # The value of the quote asset is 1 * quantity
                current_price = 1.0
                usdt_value = quantity
                print(f"Processing {asset}: Quantity={quantity:.8f}, Value={usdt_value:.2f} {quote_asset}")
            elif asset == 'BUSD':
                 # BUSD is often pegged 1:1 to USD, like USDT/USDC. Treat as 1 for approximation.
                 # In a real system, you might fetch BUSD/USDT price if needed and available.
                 print(f"Approximating BUSD value as 1 {quote_asset}")
                 current_price = 1.0
                 usdt_value = quantity
                 print(f"Processing {asset}: Quantity={quantity:.8f}, Approx Value={usdt_value:.2f} {quote_asset}")
            else:
                # Construct the symbol to fetch price (e.g., BTCUSDT)
                symbol = f"{asset}{quote_asset}"
                try:
                    ticker_info = client.get_symbol_ticker(symbol=symbol)
                    current_price = float(ticker_info['price'])
                    usdt_value = quantity * current_price
                    print(f"Processing {asset}: Quantity={quantity:.8f}, Price={current_price:.2f}, Value={usdt_value:.2f} {quote_asset}")
                    # Optional: Small delay to avoid hitting API rate limits if many assets
                    # time.sleep(0.1) 
                except Exception as e:
                    print(f"Could not fetch price for {symbol}: {e}. Skipping value calculation for this asset.")
                    current_price = 0.0 # Mark price as unknown
                    usdt_value = 0.0 # Mark value as unknown
            
            # Store calculated values
            asset_values[asset] = {
                'Quantity': quantity,
                'CurrentPrice': current_price,
                f'{quote_asset}Value': usdt_value
            }
            total_portfolio_value += usdt_value

        print(f"\n--- Approximate Total Portfolio Value: {total_portfolio_value:.2f} {quote_asset} ---")

        # Optional: Display detailed values in a DataFrame
        value_df = pd.DataFrame.from_dict(asset_values, orient='index')
        # Add percentage allocation
        if total_portfolio_value > 0:
             value_df['Allocation%'] = (value_df[f'{quote_asset}Value'] / total_portfolio_value) * 100
        else:
             value_df['Allocation%'] = 0.0

        print("\n--- Portfolio Details ---")
        print(value_df)

else:
    print("\nCannot calculate portfolio value because client failed to initialize.")

Binance client initialized successfully for value calculation.

Calculating portfolio value in USDT...
Processing BTC: Quantity=0.00042710, Price=78362.76, Value=33.47 USDT
Approximating BUSD value as 1 USDT
Processing BUSD: Quantity=0.14956400, Approx Value=0.15 USDT
Processing WAVES: Quantity=0.01000000, Price=3.33, Value=0.03 USDT

--- Approximate Total Portfolio Value: 33.65 USDT ---

--- Portfolio Details ---
       Quantity  CurrentPrice  USDTValue  Allocation%
BTC    0.000427     78362.760  33.468735    99.456626
BUSD   0.149564         1.000   0.149564     0.444449
WAVES  0.010000         3.329   0.033290     0.098925


In [15]:
import pandas as pd
import os
import numpy as np # Import numpy for NaN handling if needed

# --- Configuration ---
# DEFINE YOUR TARGET ALLOCATION PERCENTAGES HERE
# Ensure these add up to 100% !
# Include all assets you potentially want to hold, even if current balance is 0.
# Use the EXACT asset tickers (e.g., 'BTC', 'ETH', 'USDT', 'USDC')
target_allocations_pct = {
    'BTC': 70.0,   # Target 70% in Bitcoin
    'ETH': 0.0,    # Target 0% in Ether (Example)
    'USDT': 20.0,  # Target 20% in USDT (Stablecoin Reserve)
    'USDC': 0.0,   # Target 0% in USDC
    'BUSD': 10.0,  # Target 10% in BUSD (adjust based on your preference/availability)
    # Add other assets you care about, even if 0% currently
    'WAVES': 0.0   # Target 0% for WAVES
}

# --- Ensure target percentages sum to 100 ---
if abs(sum(target_allocations_pct.values()) - 100.0) > 0.01:
    print("⚠️ WARNING: Target allocation percentages do not sum close to 100%!")
    print(f"Current Sum: {sum(target_allocations_pct.values())}%")
else:
    print("Target allocations defined.")

# --- Assume 'value_df' and 'total_portfolio_value' exist from the previous cell ---
# If running independently, you would need to recalculate value_df and total_portfolio_value first.
try:
    if 'value_df' not in locals() or 'total_portfolio_value' not in locals() or total_portfolio_value <= 0:
        print("Error: 'value_df' or 'total_portfolio_value' not found or invalid.")
        print("Please run the previous portfolio value calculation cell first.")
    else:
        print("\nCalculating allocation deviation...")

        # Convert target dict to a Series for easier merging
        target_s = pd.Series(target_allocations_pct, name='Target%')

        # Combine current values and targets - use outer join to include all assets
        combined_df = value_df.join(target_s, how='outer')

        # Fill NaN values using assignment (avoids FutureWarnings)
        # Assets only in target dict get 0 current value/allocation
        combined_df['Quantity'] = combined_df['Quantity'].fillna(0)
        combined_df['CurrentPrice'] = combined_df['CurrentPrice'].fillna(0) # Or fetch price if needed
        combined_df['USDTValue'] = combined_df['USDTValue'].fillna(0)
        combined_df['Allocation%'] = combined_df['Allocation%'].fillna(0)
        # Assets only in current portfolio get 0 target allocation
        combined_df['Target%'] = combined_df['Target%'].fillna(0)

        # Recalculate total USDT value from combined df in case new assets were added
        total_portfolio_value = combined_df['USDTValue'].sum()

        # Recalculate actual Allocation% based on potentially updated total
        if total_portfolio_value > 0:
             combined_df['Allocation%'] = (combined_df['USDTValue'] / total_portfolio_value) * 100
        else:
             combined_df['Allocation%'] = 0.0

        # Calculate Deviation
        combined_df['Deviation%'] = combined_df['Allocation%'] - combined_df['Target%']

        # Calculate Target USDT Value and Deviation in USDT
        combined_df['TargetValue'] = (combined_df['Target%'] / 100.0) * total_portfolio_value
        combined_df['DeviationValue'] = combined_df['USDTValue'] - combined_df['TargetValue']

        print("\n--- Portfolio Allocation vs Target ---")
        # Display relevant columns, rounded for clarity
        display_cols = ['USDTValue', 'Allocation%', 'Target%', 'TargetValue', 'Deviation%', 'DeviationValue']
        print(combined_df[display_cols].round(2))

        print(f"\nTotal Portfolio Value: {total_portfolio_value:.2f} USDT")

except NameError:
     print("Error: 'value_df' or 'total_portfolio_value' variable not found.")
     print("Please run the previous portfolio value calculation cell first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Target allocations defined.

Calculating allocation deviation...

--- Portfolio Allocation vs Target ---
       USDTValue  Allocation%  Target%  TargetValue  Deviation%  \
BTC        33.47        99.46     70.0        23.56       29.46   
BUSD        0.15         0.44     10.0         3.37       -9.56   
ETH         0.00         0.00      0.0         0.00        0.00   
USDC        0.00         0.00      0.0         0.00        0.00   
USDT        0.00         0.00     20.0         6.73      -20.00   
WAVES       0.03         0.10      0.0         0.00        0.10   

       DeviationValue  
BTC              9.91  
BUSD            -3.22  
ETH              0.00  
USDC             0.00  
USDT            -6.73  
WAVES            0.03  

Total Portfolio Value: 33.65 USDT


In [17]:
import pandas as pd
import os
from binance.client import Client # Make sure this library is installed
from dotenv import load_dotenv     # Make sure this library is installed

# --- Load API Keys and Initialize Client ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None # Initialize client as None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping() # Quick check
        print("Binance client initialized successfully for ticker data.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Configuration ---
quote_asset_filter = 'USDT' # Focus on pairs quoted in USDT

# --- Fetch and Process Ticker Data ---
if client:
    try:
        print(f"\nFetching 24hr ticker data for all symbols...")
        all_tickers = client.get_ticker() # Get data for ALL symbols
        print(f"Received ticker data for {len(all_tickers)} symbols.")

        # Filter for pairs ending with our quote asset
        usdt_tickers = [
            ticker for ticker in all_tickers 
            if ticker.get('symbol', '').endswith(quote_asset_filter)
        ]
        print(f"Filtered down to {len(usdt_tickers)} {quote_asset_filter} pairs.")

        # Extract relevant information
        volume_data = []
        for ticker in usdt_tickers:
            symbol = ticker.get('symbol')
            base_asset = symbol.replace(quote_asset_filter, '') # Extract base asset
            # 'quoteVolume' is typically the 24hr volume in the quote asset (USDT)
            volume = float(ticker.get('quoteVolume', 0.0)) 
            last_price = float(ticker.get('lastPrice', 0.0))
            
            volume_data.append({
                'Asset': base_asset,
                'Symbol': symbol,
                f'Volume_{quote_asset_filter}': volume,
                'LastPrice': last_price
            })
        
        # Create DataFrame
        volume_df = pd.DataFrame(volume_data)
        
        # Sort by volume descending
        volume_df.sort_values(by=f'Volume_{quote_asset_filter}', ascending=False, inplace=True)
        
        # Set Asset as index
        # volume_df.set_index('Asset', inplace=True) # Optional: Keep as column for now

        print(f"\n--- Top 15 Assets by 24hr {quote_asset_filter} Volume on Binance.US ---")
        print(volume_df.head(15)) # Display top N

    except Exception as e:
        print(f"An error occurred while fetching or processing ticker data: {e}")
        volume_df = pd.DataFrame() # Ensure volume_df exists but is empty on error

else:
    print("\nCannot fetch ticker data because client failed to initialize.")
    volume_df = pd.DataFrame() # Ensure volume_df exists but is empty

Binance client initialized successfully for ticker data.

Fetching 24hr ticker data for all symbols...
Received ticker data for 573 symbols.
Filtered down to 188 USDT pairs.

--- Top 15 Assets by 24hr USDT Volume on Binance.US ---
    Asset    Symbol   Volume_USDT     LastPrice
2     XRP   XRPUSDT  5.359587e+06      1.880500
0     BTC   BTCUSDT  4.215311e+06  78413.550000
1     ETH   ETHUSDT  2.869413e+06   1551.870000
27    SOL   SOLUSDT  1.220143e+06    106.150000
6     ADA   ADAUSDT  6.466906e+05      0.573200
166   SUI   SUIUSDT  5.596091e+05      2.032400
11   DOGE  DOGEUSDT  3.666854e+05      0.147520
154  HBAR  HBARUSDT  3.000812e+05      0.147090
5     BNB   BNBUSDT  2.858852e+05    552.180000
44   USDC  USDCUSDT  2.138661e+05      1.001000
4     LTC   LTCUSDT  1.877404e+05     70.900000
9     XLM   XLMUSDT  1.827071e+05      0.230500
35   SHIB  SHIBUSDT  1.119952e+05      0.000011
63   GALA  GALAUSDT  1.028687e+05      0.013340
48   LINK  LINKUSDT  1.005829e+05     11.300000


In [18]:
import pandas as pd
import os
import numpy as np

# --- Configuration ---
TOP_N_ASSETS = 10 # How many top volume assets to include (excluding stablecoins)
STABLECOIN_RESERVE_PCT = 20.0 # Desired % allocation to stablecoins
STABLECOIN_ASSETS = ['USDT', 'USDC', 'BUSD'] # Define which assets count as stablecoins

# --- Assume 'volume_df' exists from the previous cell ---
try:
    if 'volume_df' not in locals() or volume_df.empty:
        print("Error: 'volume_df' not found or is empty.")
        print("Please run the previous ticker data fetching cell first.")
        dynamic_target_allocations_pct = {} # Initialize empty dict on error
    else:
        print(f"Calculating dynamic targets: Top {TOP_N_ASSETS} by volume + {STABLECOIN_RESERVE_PCT}% stablecoins.")
        
        # Filter out stablecoins from the volume ranking for Top N selection
        crypto_volume_df = volume_df[~volume_df['Asset'].isin(STABLECOIN_ASSETS)].copy()
        
        # Select Top N crypto assets
        top_n_crypto_df = crypto_volume_df.head(TOP_N_ASSETS)
        
        # Calculate the total volume of these Top N crypto assets
        total_top_n_volume = top_n_crypto_df[f'Volume_{quote_asset_filter}'].sum()
        print(f"Total 24hr Volume for Top {TOP_N_ASSETS} crypto assets: {total_top_n_volume:.2f} {quote_asset_filter}")

        # --- Calculate Dynamic Allocations ---
        dynamic_target_allocations_pct = {}
        crypto_allocation_pct = 100.0 - STABLECOIN_RESERVE_PCT 
        
        if total_top_n_volume > 0:
            # Calculate allocation for each Top N crypto asset based on volume proportion
            for index, row in top_n_crypto_df.iterrows():
                asset = row['Asset']
                volume = row[f'Volume_{quote_asset_filter}']
                # Proportion of this asset's volume within the Top N * available crypto allocation %
                target_pct = (volume / total_top_n_volume) * crypto_allocation_pct
                dynamic_target_allocations_pct[asset] = target_pct
        else:
            print("Warning: Total volume for Top N crypto assets is zero. Cannot allocate crypto percentages.")

        # --- Add Stablecoin Allocation ---
        # For simplicity, allocate the entire reserve to the primary quote asset (USDT)
        # Could be split among multiple stablecoins if desired
        primary_stablecoin = quote_asset_filter # Usually USDT
        if primary_stablecoin in dynamic_target_allocations_pct:
             # If USDT somehow made it into top N crypto (unlikely), add reserve to it
             dynamic_target_allocations_pct[primary_stablecoin] += STABLECOIN_RESERVE_PCT
        else:
             dynamic_target_allocations_pct[primary_stablecoin] = STABLECOIN_RESERVE_PCT
             
        # Ensure other defined stablecoins have a 0% target if not the primary one chosen
        for stable in STABLECOIN_ASSETS:
            if stable != primary_stablecoin and stable not in dynamic_target_allocations_pct:
                dynamic_target_allocations_pct[stable] = 0.0


        # --- Display Calculated Targets ---
        print("\n--- Dynamically Calculated Target Allocations (%) ---")
        # Sort for better readability (optional)
        sorted_targets = dict(sorted(dynamic_target_allocations_pct.items(), key=lambda item: item[1], reverse=True))
        for asset, pct in sorted_targets.items():
            print(f"{asset}: {pct:.2f}%")

        # --- Sanity Check: Sum of dynamic targets ---
        total_dynamic_pct = sum(dynamic_target_allocations_pct.values())
        print(f"\nSum of dynamic target allocations: {total_dynamic_pct:.2f}%")
        if abs(total_dynamic_pct - 100.0) > 0.01:
             print("⚠️ WARNING: Dynamic target allocations do not sum close to 100%!")


except NameError:
     print("Error: 'volume_df' variable not found.")
     print("Please run the previous ticker data fetching cell first.")
     dynamic_target_allocations_pct = {} # Initialize empty dict on error
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    dynamic_target_allocations_pct = {} # Initialize empty dict on error

Calculating dynamic targets: Top 10 by volume + 20.0% stablecoins.
Total 24hr Volume for Top 10 crypto assets: 16011146.45 USDT

--- Dynamically Calculated Target Allocations (%) ---
XRP: 26.78%
BTC: 21.06%
USDT: 20.00%
ETH: 14.34%
SOL: 6.10%
ADA: 3.23%
SUI: 2.80%
DOGE: 1.83%
HBAR: 1.50%
BNB: 1.43%
LTC: 0.94%
USDC: 0.00%
BUSD: 0.00%

Sum of dynamic target allocations: 100.00%


In [19]:
import pandas as pd
import os
import numpy as np
from binance.client import Client # Make sure this library is installed
from dotenv import load_dotenv     # Make sure this library is installed
import time # Import time for potential rate limiting delays

# --- Load API Keys and Initialize Client (Needed for current value) ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None # Initialize client as None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping() # Quick check
        print("Binance client initialized successfully for deviation calc.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Function Definition (assuming get_current_balances exists or include it) ---
# Re-include the function to make this cell self-contained
def get_current_balances(api_client):
    if not api_client: return pd.DataFrame(columns=['Asset', 'Free', 'Locked']).set_index('Asset')
    try:
        account_info = api_client.get_account()
        balances_raw = account_info.get('balances', [])
        processed_balances = []
        for asset_info in balances_raw:
            free = float(asset_info['free'])
            locked = float(asset_info['locked'])
            if free > 0 or locked > 0:
                processed_balances.append({'Asset': asset_info['asset'], 'Free': free, 'Locked': locked})
        if not processed_balances: return pd.DataFrame(columns=['Asset', 'Free', 'Locked']).set_index('Asset')
        balances_df = pd.DataFrame(processed_balances)
        balances_df.set_index('Asset', inplace=True)
        return balances_df
    except Exception as e:
        print(f"Error fetching account balances: {e}")
        return pd.DataFrame(columns=['Asset', 'Free', 'Locked']).set_index('Asset')

# --- Get Current Portfolio Value (Recalculate for accuracy) ---
current_value_df = pd.DataFrame() # Initialize empty df
total_portfolio_value_now = 0.0
quote_asset = 'USDT' # Match quote asset from previous steps

if client:
    balances_df_now = get_current_balances(client)
    if not balances_df_now.empty:
        print(f"\nRecalculating current portfolio value in {quote_asset}...")
        asset_values_now = {}
        for asset, row in balances_df_now.iterrows():
            quantity = row['Free'] + row['Locked']
            current_price = 0.0
            usdt_value = 0.0
            if asset == quote_asset:
                current_price = 1.0
                usdt_value = quantity
            elif asset == 'BUSD': # Approximation
                 current_price = 1.0
                 usdt_value = quantity
            else:
                symbol = f"{asset}{quote_asset}"
                try:
                    ticker_info = client.get_symbol_ticker(symbol=symbol)
                    current_price = float(ticker_info['price'])
                    usdt_value = quantity * current_price
                    # time.sleep(0.1) # Optional delay
                except Exception as e:
                    print(f"Could not fetch price for {symbol} (for current value): {e}.")
                    current_price = 0.0
                    usdt_value = 0.0
            asset_values_now[asset] = {
                'Quantity': quantity, 'CurrentPrice': current_price, f'{quote_asset}Value': usdt_value
            }
            total_portfolio_value_now += usdt_value
        
        current_value_df = pd.DataFrame.from_dict(asset_values_now, orient='index')
        if total_portfolio_value_now > 0:
             current_value_df['Allocation%'] = (current_value_df[f'{quote_asset}Value'] / total_portfolio_value_now) * 100
        else:
             current_value_df['Allocation%'] = 0.0
        print(f"Current Total Portfolio Value: {total_portfolio_value_now:.2f} {quote_asset}")
    else:
         print("Could not retrieve current balances to calculate value.")
else:
    print("Client not initialized, cannot calculate current portfolio value.")


# --- Assume 'dynamic_target_allocations_pct' exists from the previous cell ---
try:
    # Check if dynamic targets and current value were obtained
    if 'dynamic_target_allocations_pct' not in locals() or not dynamic_target_allocations_pct:
        print("\nError: 'dynamic_target_allocations_pct' not found or is empty.")
        print("Please run the previous dynamic target calculation cell first.")
    elif current_value_df.empty or total_portfolio_value_now <= 0:
        print("\nError: Current portfolio value could not be determined.")
        print("Cannot calculate deviation without current portfolio state.")
    else:
        print("\nCalculating allocation deviation vs DYNAMIC targets...")

        # Convert dynamic target dict to a Series
        target_s = pd.Series(dynamic_target_allocations_pct, name='Target%')

        # Combine current values and dynamic targets
        combined_df = current_value_df.join(target_s, how='outer')

        # Fill NaN values using assignment
        combined_df['Quantity'] = combined_df['Quantity'].fillna(0)
        combined_df['CurrentPrice'] = combined_df['CurrentPrice'].fillna(0)
        combined_df['USDTValue'] = combined_df['USDTValue'].fillna(0)
        combined_df['Allocation%'] = combined_df['Allocation%'].fillna(0)
        combined_df['Target%'] = combined_df['Target%'].fillna(0)

        # Re-sum total value and re-calculate Allocation% after merge/fillna
        total_portfolio_value_final = combined_df['USDTValue'].sum()
        if total_portfolio_value_final > 0:
             combined_df['Allocation%'] = (combined_df['USDTValue'] / total_portfolio_value_final) * 100
        else:
             combined_df['Allocation%'] = 0.0

        # Calculate Deviation against DYNAMIC targets
        combined_df['Deviation%'] = combined_df['Allocation%'] - combined_df['Target%']
        combined_df['TargetValue'] = (combined_df['Target%'] / 100.0) * total_portfolio_value_final
        combined_df['DeviationValue'] = combined_df['USDTValue'] - combined_df['TargetValue']

        print("\n--- Portfolio Allocation vs DYNAMIC Target ---")
        # Display relevant columns, rounded for clarity
        display_cols = ['USDTValue', 'Allocation%', 'Target%', 'TargetValue', 'Deviation%', 'DeviationValue']
        print(combined_df[display_cols].round(2))

        print(f"\nTotal Portfolio Value: {total_portfolio_value_final:.2f} USDT")

except NameError as e:
     print(f"\nError: A required variable was not found ({e}).")
     print("Please ensure the previous cells defining dynamic targets and calculating portfolio value have been run.")
except Exception as e:
    print(f"\nAn unexpected error occurred during deviation calculation: {e}")

Binance client initialized successfully for deviation calc.

Recalculating current portfolio value in USDT...
Current Total Portfolio Value: 33.60 USDT

Calculating allocation deviation vs DYNAMIC targets...

--- Portfolio Allocation vs DYNAMIC Target ---
       USDTValue  Allocation%  Target%  TargetValue  Deviation%  \
ADA         0.00         0.00     3.23         1.09       -3.23   
BNB         0.00         0.00     1.43         0.48       -1.43   
BTC        33.42        99.46    21.06         7.08       78.39   
BUSD        0.15         0.45     0.00         0.00        0.45   
DOGE        0.00         0.00     1.83         0.62       -1.83   
ETH         0.00         0.00    14.34         4.82      -14.34   
HBAR        0.00         0.00     1.50         0.50       -1.50   
LTC         0.00         0.00     0.94         0.32       -0.94   
SOL         0.00         0.00     6.10         2.05       -6.10   
SUI         0.00         0.00     2.80         0.94       -2.80   
USDC   

In [22]:
import pandas as pd
import os
import numpy as np
from binance.client import Client # Make sure library is installed
from dotenv import load_dotenv     # Make sure library is installed
import time                        # Import time for potential delays

# --- Configuration ---
TRADE_THRESHOLD_USDT = 0.50 
STABLECOIN_ASSETS = ['USDT', 'USDC', 'BUSD'] 
# Define quote asset again for price fetching
quote_asset = 'USDT'

# --- Initialize Client (Needed for fetching prices) ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')
client = None 
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping() 
        print("Binance client initialized successfully for trade determination.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")


# --- Assume 'combined_df' exists from the previous deviation calculation cell ---
try:
    if not client:
         raise ConnectionError("Binance client not initialized. Cannot proceed.")
         
    if 'combined_df' not in locals() or combined_df.empty:
        print("Error: 'combined_df' not found or is empty.")
        print("Please run the previous deviation calculation cell first.")
    else:
        print("\nDetermining necessary rebalancing trades...")

        # Identify assets to SELL 
        sell_candidates = combined_df[combined_df['DeviationValue'] > TRADE_THRESHOLD_USDT].copy()

        # Identify assets to BUY 
        buy_candidates = combined_df[combined_df['DeviationValue'] < -TRADE_THRESHOLD_USDT].copy()

        # --- Fetch Prices for Buy Candidates if Missing ---
        print("Checking/Fetching prices for assets to BUY...")
        for asset in buy_candidates.index:
            # Check if price is missing (0 or NaN) and it's not a stablecoin we approximate
            if buy_candidates.loc[asset, 'CurrentPrice'] <= 0 and asset not in STABLECOIN_ASSETS:
                symbol = f"{asset}{quote_asset}"
                try:
                    ticker_info = client.get_symbol_ticker(symbol=symbol)
                    fetched_price = float(ticker_info['price'])
                    if fetched_price > 0:
                         buy_candidates.loc[asset, 'CurrentPrice'] = fetched_price
                         print(f"Fetched price for {asset}: {fetched_price}")
                    else:
                         print(f"Warning: Fetched price for {symbol} is zero or invalid.")
                    # time.sleep(0.1) # Optional delay
                except Exception as e:
                    print(f"Could not fetch price for {symbol} (buy candidate): {e}. Quantity will be 0.")
            # Ensure stablecoins we approximate have price 1.0
            elif asset in STABLECOIN_ASSETS and buy_candidates.loc[asset, 'CurrentPrice'] <= 0:
                 buy_candidates.loc[asset, 'CurrentPrice'] = 1.0
                 print(f"Set price for stablecoin {asset} to 1.0")


        # --- Calculate Trade Quantities (Approximate) ---
        proposed_trades = []

        # Calculate SELL quantities
        if not sell_candidates.empty:
            print("\nAssets to potentially SELL:")
            sell_candidates['SellQuantity'] = 0.0
            sell_candidates['SellQuantity'] = sell_candidates.apply(
                lambda row: row['DeviationValue'] / row['CurrentPrice'] if row['CurrentPrice'] > 0 else 0.0,
                axis=1
            ).astype(float)
            print(sell_candidates[['DeviationValue', 'CurrentPrice', 'SellQuantity']].round(6))

            for asset, row in sell_candidates.iterrows():
                 if row['SellQuantity'] > 0:
                    proposed_trades.append({
                        'Action': 'SELL', 'Asset': asset,
                        'ApproxQuantity': row['SellQuantity'], 'ApproxValue': row['DeviationValue']
                    })

        # Calculate BUY quantities (using potentially updated prices)
        if not buy_candidates.empty:
            print("\nAssets to potentially BUY:")
            buy_candidates['BuyQuantity'] = 0.0
            buy_candidates['BuyQuantity'] = buy_candidates.apply(
                lambda row: abs(row['DeviationValue']) / row['CurrentPrice'] if row['CurrentPrice'] > 0 else 0.0,
                axis=1
            ).astype(float)
            
            # Special handling for stablecoin quantity remains valid as price is now 1.0
            stable_mask = buy_candidates.index.isin(STABLECOIN_ASSETS)
            buy_candidates.loc[stable_mask, 'BuyQuantity'] = abs(buy_candidates.loc[stable_mask, 'DeviationValue'])


            print(buy_candidates[['DeviationValue', 'CurrentPrice', 'BuyQuantity']].round(6))

            for asset, row in buy_candidates.iterrows():
                 if row['BuyQuantity'] > 0:
                    proposed_trades.append({
                        'Action': 'BUY', 'Asset': asset,
                        'ApproxQuantity': row['BuyQuantity'], 'ApproxValue': abs(row['DeviationValue'])
                    })

        # --- Display Proposed Trades ---
        print("\n--- Proposed Rebalancing Trades ---")
        if not proposed_trades:
            print("No significant deviations found. Portfolio is balanced according to targets.")
        else:
            trades_df = pd.DataFrame(proposed_trades)
            trades_df = trades_df[['Action', 'Asset', 'ApproxQuantity', 'ApproxValue']]
            print(trades_df.round({'ApproxQuantity': 8, 'ApproxValue': 2}))

            total_sell_value = trades_df[trades_df['Action'] == 'SELL']['ApproxValue'].sum()
            total_buy_value = trades_df[trades_df['Action'] == 'BUY']['ApproxValue'].sum()
            print(f"\nApproximate Total SELL Value: {total_sell_value:.2f} USDT")
            print(f"Approximate Total BUY Value : {total_buy_value:.2f} USDT")

except NameError as ne:
     print(f"Error: A required variable was not found ({ne}).")
     print("Please ensure the previous cells defining deviation have been run.")
except ConnectionError as ce:
     print(f"Connection Error: {ce}") # Handle specific client init error
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Binance client initialized successfully for trade determination.

Determining necessary rebalancing trades...
Checking/Fetching prices for assets to BUY...
Fetched price for ADA: 0.5773
Fetched price for DOGE: 0.14805
Fetched price for ETH: 1558.41
Fetched price for HBAR: 0.14839
Fetched price for SOL: 106.24
Fetched price for SUI: 2.0173
Set price for stablecoin USDT to 1.0
Fetched price for XRP: 1.8944

Assets to potentially SELL:
     DeviationValue  CurrentPrice  SellQuantity
BTC       26.338913       78237.7      0.000337

Assets to potentially BUY:
      DeviationValue  CurrentPrice  BuyQuantity
ADA        -1.085625       0.57730     1.880521
DOGE       -0.615569       0.14805     4.157847
ETH        -4.816996    1558.41000     0.003091
HBAR       -0.503758       0.14839     3.394825
SOL        -2.048302     106.24000     0.019280
SUI        -0.939438       2.01730     0.465691
USDT       -6.719635       1.00000     6.719635
XRP        -8.997350       1.89440     4.749446

--- Pr

In [24]:
import pandas as pd
import os
import numpy as np
from binance.client import Client # Make sure library is installed
from dotenv import load_dotenv     # Make sure library is installed
from decimal import Decimal # Use Decimal for precision handling

# --- Load API Keys and Initialize Client (Needed for exchange info) ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None # Initialize client as None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping() # Quick check
        print("Binance client initialized successfully for order formatting.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Configuration ---
quote_asset = 'USDT'

# --- Helper function to adjust quantity according to LOT_SIZE filter ---
def adjust_quantity_to_step(quantity, step_size_str):
    """Adjusts quantity down to the nearest valid step size."""
    step_size = Decimal(step_size_str)
    quantity_decimal = Decimal(str(quantity)) # Convert float to Decimal string first
    # Calculate remainder, subtract it to round down
    remainder = quantity_decimal.remainder_near(step_size) 
    adjusted_quantity = quantity_decimal - remainder # Simple floor based on step
    # More precise floor:
    adjusted_quantity = (quantity_decimal // step_size) * step_size
    return float(adjusted_quantity) # Return as float for API

# --- Assume 'trades_df' exists from the previous step ---
try:
    if not client:
         raise ConnectionError("Binance client not initialized. Cannot proceed.")
         
    if 'trades_df' not in locals() or trades_df.empty:
        print("Error: 'trades_df' not found or is empty.")
        print("Please run the previous trade determination cell first.")
    else:
        print("\nFormatting proposed trades into executable order parameters...")
        
        # --- Get Exchange Info (Contains Rules/Filters) ---
        print("Fetching exchange information (for order rules)...")
        exchange_info = client.get_exchange_info()
        symbols_info = {item['symbol']: item for item in exchange_info['symbols']}
        print("Exchange information received.")

        executable_orders = [] # List to hold formatted orders that pass checks

        for index, trade in trades_df.iterrows():
            action = trade['Action']
            asset = trade['Asset']
            quantity = trade['ApproxQuantity']
            value = trade['ApproxValue']
            
            # Skip trying to trade the quote asset itself (no pair like USDTUSDT)
            if asset == quote_asset:
                 print(f"Skipping trade for quote asset {asset}")
                 continue
                 
            symbol = f"{asset}{quote_asset}"

            print(f"\nProcessing: {action} {quantity:.8f} {asset} ({symbol})")

            # --- Check if symbol exists and get its rules ---
            if symbol not in symbols_info:
                print(f"Warning: Symbol {symbol} not found in exchange info. Cannot format order.")
                continue
                
            symbol_info = symbols_info[symbol]
            filters = {f['filterType']: f for f in symbol_info['filters']}

            # --- 1. Adjust Quantity Precision (LOT_SIZE) ---
            adjusted_quantity = quantity
            if 'LOT_SIZE' in filters:
                step_size = filters['LOT_SIZE'].get('stepSize')
                if step_size:
                    original_qty = adjusted_quantity
                    adjusted_quantity = adjust_quantity_to_step(adjusted_quantity, step_size)
                    if adjusted_quantity != original_qty:
                         print(f"Adjusted quantity for LOT_SIZE (step {step_size}): {original_qty:.8f} -> {adjusted_quantity:.8f}")
                    if adjusted_quantity <= 0:
                         print("Adjusted quantity is zero or less after applying LOT_SIZE. Skipping trade.")
                         continue
                else:
                     print(f"Warning: stepSize not found in LOT_SIZE filter for {symbol}")
            else:
                print(f"Warning: LOT_SIZE filter not found for {symbol}")

            # --- 2. Check Minimum Order Requirements ---
            min_notional = 0.0
            min_qty = 0.0
            current_price = trade.get('CurrentPrice') # Get price if available from previous step
            # If price wasn't passed, fetch it again (needed for MIN_NOTIONAL)
            if not current_price or current_price <= 0:
                 try:
                      ticker_info = client.get_symbol_ticker(symbol=symbol)
                      current_price = float(ticker_info['price'])
                 except Exception as e:
                      print(f"Could not fetch price for {symbol} to check MIN_NOTIONAL. Skipping trade. Error: {e}")
                      continue
            
            order_value = adjusted_quantity * current_price

            # Check MIN_NOTIONAL (most common)
            if 'MIN_NOTIONAL' in filters:
                min_notional = float(filters['MIN_NOTIONAL'].get('minNotional', 0.0))
                if order_value < min_notional:
                    print(f"Order value ({order_value:.4f}) is below MIN_NOTIONAL ({min_notional:.4f}). Skipping trade.")
                    continue
            # Check MARKET_LOT_SIZE (minQty - less common for value check, but good to know)
            elif 'MARKET_LOT_SIZE' in filters:
                 min_qty = float(filters['MARKET_LOT_SIZE'].get('minQty', 0.0))
                 if adjusted_quantity < min_qty:
                      print(f"Order quantity ({adjusted_quantity:.8f}) is below MARKET_LOT_SIZE minQty ({min_qty:.8f}). Skipping trade.")
                      continue
            # Check deprecated LOT_SIZE minQty as fallback if others missing
            elif 'LOT_SIZE' in filters:
                 min_qty = float(filters['LOT_SIZE'].get('minQty', 0.0))
                 if adjusted_quantity < min_qty:
                      print(f"Order quantity ({adjusted_quantity:.8f}) is below LOT_SIZE minQty ({min_qty:.8f}). Skipping trade.")
                      continue
            else:
                print(f"Warning: No MIN_NOTIONAL or relevant LOT_SIZE filter found for {symbol}. Proceeding without min check.")


            # --- If all checks pass, format the order ---
            print(f"Order for {symbol} meets requirements. Adding to list.")
            executable_orders.append({
                'symbol': symbol,
                'side': action, # 'BUY' or 'SELL'
                'type': 'MARKET', # Use MARKET orders for simplicity in rebalancing
                'quantity': adjusted_quantity 
                # Note: For MARKET SELL using quantity, API handles it.
                # For MARKET BUY, sometimes 'quoteOrderQty' (USDT amount) is preferred/required
                # We'll stick with quantity for now, but be aware of this nuance.
            })

        # --- Display Formatted Orders ---
        print("\n--- Potential Orders Ready for Execution (Simulation) ---")
        if not executable_orders:
            print("No trades met the exchange requirements after filtering.")
        else:
            orders_df = pd.DataFrame(executable_orders)
            print(orders_df)

except NameError as ne:
     print(f"\nError: A required variable was not found ({ne}).")
     print("Please ensure the previous cells have been run.")
except ConnectionError as ce:
     print(f"Connection Error: {ce}") 
except Exception as e:
    print(f"\nAn unexpected error occurred during order formatting: {e}")

Binance client initialized successfully for order formatting.

Formatting proposed trades into executable order parameters...
Fetching exchange information (for order rules)...
Exchange information received.

Processing: SELL 0.00033665 BTC (BTCUSDT)
Adjusted quantity for LOT_SIZE (step 0.00001000): 0.00033665 -> 0.00033000
Order for BTCUSDT meets requirements. Adding to list.

Processing: BUY 1.88052132 ADA (ADAUSDT)
Adjusted quantity for LOT_SIZE (step 0.10000000): 1.88052132 -> 1.80000000
Order for ADAUSDT meets requirements. Adding to list.

Processing: BUY 4.15784685 DOGE (DOGEUSDT)
Adjusted quantity for LOT_SIZE (step 1.00000000): 4.15784685 -> 4.00000000
Order value (0.5911) is below MIN_NOTIONAL (1.0000). Skipping trade.

Processing: BUY 0.00309097 ETH (ETHUSDT)
Adjusted quantity for LOT_SIZE (step 0.00010000): 0.00309097 -> 0.00300000
Order for ETHUSDT meets requirements. Adding to list.

Processing: BUY 3.39482466 HBAR (HBARUSDT)
Adjusted quantity for LOT_SIZE (step 1.0000000

In [26]:
import pandas as pd
import os
from binance.client import Client # Make sure library is installed
from dotenv import load_dotenv     # Make sure library is installed

# --- Load API Keys and Initialize Client ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None # Initialize client as None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping() # Quick check
        print("Binance client initialized successfully for test orders.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Assume 'executable_orders' list exists from the previous step ---
try:
    if not client:
         raise ConnectionError("Binance client not initialized. Cannot proceed.")
         
    if 'executable_orders' not in locals() or not executable_orders:
        print("Error: 'executable_orders' list not found or is empty.")
        print("Please run the previous order formatting cell first, or no orders met requirements.")
    else:
        print("\n--- Sending Test Orders (Simulation - No Real Trades) ---")
        
        successful_tests = 0
        failed_tests = 0

        for order_params in executable_orders:
            print(f"Attempting TEST {order_params['side']} for {order_params['quantity']} {order_params['symbol']}...")
            try:
                # Send the test order
                test_result = client.create_test_order(
                    symbol=order_params['symbol'],
                    side=order_params['side'],
                    type=order_params['type'], # Should be 'MARKET'
                    quantity=order_params['quantity']
                    # For MARKET BUY, consider quoteOrderQty if quantity causes issues
                )
                
                # A successful test order returns an empty dictionary {}
                if isinstance(test_result, dict) and not test_result:
                    print(f"  ✅ SUCCESS: Test order for {order_params['symbol']} validated.")
                    successful_tests += 1
                else:
                    # This case shouldn't typically happen for successful tests, but include for safety
                    print(f"  ⚠️ UNEXPECTED RESULT (Expected empty dict): {test_result}")
                    failed_tests += 1

            except Exception as e:
                print(f"  ❌ FAILED: Test order for {order_params['symbol']} failed validation.")
                print(f"     Error: {e}")
                failed_tests += 1

        print("\n--- Test Order Summary ---")
        print(f"Successful Validations: {successful_tests}")
        print(f"Failed Validations    : {failed_tests}")
        
        if failed_tests > 0:
            print("\nReview failed test orders and adjust parameters or logic before attempting real trades.")

except NameError as ne:
     print(f"\nError: A required variable was not found ({ne}).")
     print("Please ensure the previous cells have been run.")
except ConnectionError as ce:
     print(f"Connection Error: {ce}") 
except Exception as e:
    print(f"\nAn unexpected error occurred during test order simulation: {e}")

Binance client initialized successfully for test orders.

--- Sending Test Orders (Simulation - No Real Trades) ---
Attempting TEST SELL for 0.00033 BTCUSDT...
  ✅ SUCCESS: Test order for BTCUSDT validated.
Attempting TEST BUY for 1.8 ADAUSDT...
  ✅ SUCCESS: Test order for ADAUSDT validated.
Attempting TEST BUY for 0.003 ETHUSDT...
  ✅ SUCCESS: Test order for ETHUSDT validated.
Attempting TEST BUY for 0.019 SOLUSDT...
  ✅ SUCCESS: Test order for SOLUSDT validated.
Attempting TEST BUY for 4.0 XRPUSDT...
  ✅ SUCCESS: Test order for XRPUSDT validated.

--- Test Order Summary ---
Successful Validations: 5
Failed Validations    : 0


In [27]:
import pandas as pd
import os
import numpy as np
from binance.client import Client 
from dotenv import load_dotenv     
from decimal import Decimal, ROUND_DOWN, ROUND_UP # Use Decimal for precision

# --- Load API Keys and Initialize Client ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None 
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping() 
        print("Binance client initialized successfully for LIMIT order formatting.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Configuration ---
quote_asset = 'USDT'
# Define stablecoins again if needed for logic (not directly used in price setting here)
STABLECOIN_ASSETS = ['USDT', 'USDC', 'BUSD'] 

# --- Helper functions for precision ---
def adjust_quantity_to_step(quantity, step_size_str):
    step_size = Decimal(step_size_str)
    quantity_decimal = Decimal(str(quantity))
    adjusted_quantity = (quantity_decimal // step_size) * step_size # Floor division
    return float(adjusted_quantity) 

def adjust_price_to_tick(price, tick_size_str, side):
     tick_size = Decimal(tick_size_str)
     price_decimal = Decimal(str(price))
     # For BUY, round down to nearest tick to ensure it's placeable below ask
     # For SELL, round up to nearest tick to ensure it's placeable above bid
     # However, rounding needs care. Let's just format to the required decimal places.
     # The API often rejects prices with too many decimals.
     # Correct rounding requires quantizing.
     # Floor for BUY, Ceil for SELL relative to tick_size
     if side == 'BUY':
         adjusted_price = (price_decimal // tick_size) * tick_size
     elif side == 'SELL':
         # Ceiling requires a bit more care with Decimal
         adjusted_price = ((price_decimal + tick_size - Decimal('1e-10')) // tick_size) * tick_size # Add tiny amount before floor for ceiling effect
         # A simpler approach for formatting decimals (might not be perfect rounding):
         # Calculate number of decimal places in tick_size
         decimals = abs(tick_size.as_tuple().exponent)
         adjusted_price = price_decimal.quantize(Decimal('1e-' + str(decimals)), rounding=ROUND_DOWN if side=='BUY' else ROUND_UP)

     else: # Default case or error
         adjusted_price = price_decimal 
         
     # Let's stick to quantize for now as it's clearer for decimal places
     decimals = abs(tick_size.as_tuple().exponent)
     rounding_mode = ROUND_DOWN if side == 'BUY' else ROUND_UP
     adjusted_price = price_decimal.quantize(Decimal('1e-' + str(decimals)), rounding=rounding_mode)

     return float(adjusted_price)


# --- Assume 'trades_df' exists from the previous step ---
try:
    if not client:
         raise ConnectionError("Binance client not initialized. Cannot proceed.")
         
    if 'trades_df' not in locals() or trades_df.empty:
        print("Error: 'trades_df' not found or is empty.")
        print("Please run the previous trade determination cell first.")
    else:
        print("\nFormatting proposed trades into LIMIT order parameters (Maker Strategy)...")
        
        # --- Get Exchange Info ---
        print("Fetching exchange information...")
        exchange_info = client.get_exchange_info()
        symbols_info = {item['symbol']: item for item in exchange_info['symbols']}
        print("Exchange information received.")

        limit_orders_to_test = [] 

        for index, trade in trades_df.iterrows():
            action = trade['Action'] # 'BUY' or 'SELL'
            asset = trade['Asset']
            quantity = trade['ApproxQuantity']
            
            if asset == quote_asset: continue # Skip quote asset
                 
            symbol = f"{asset}{quote_asset}"
            print(f"\nProcessing: {action} {quantity:.8f} {asset} ({symbol})")

            if symbol not in symbols_info:
                print(f"Warning: Symbol {symbol} not found. Skipping.")
                continue
                
            symbol_info = symbols_info[symbol]
            filters = {f['filterType']: f for f in symbol_info['filters']}

            # --- 1. Get Order Book (Best Bid/Ask) ---
            try:
                 # Fetch top 1 order book entry (0=best bid, 0=best ask)
                 depth = client.get_order_book(symbol=symbol, limit=5) 
                 best_bid = float(depth['bids'][0][0]) # Highest price someone is willing to buy at
                 best_ask = float(depth['asks'][0][0]) # Lowest price someone is willing to sell at
                 print(f"  Best Bid: {best_bid}, Best Ask: {best_ask}")
            except Exception as e:
                 print(f"  Could not fetch order book for {symbol}: {e}. Skipping.")
                 continue

            # --- 2. Determine Target Limit Price (Simple Maker Strategy) ---
            limit_price = 0.0
            if action == 'BUY':
                # Place buy slightly BELOW current best ask (or at it, hoping it moves)
                # Using best_ask directly often results in taker if spread is tight
                # Let's target the best_ask for this simple strategy first
                limit_price = best_ask 
                # More aggressive maker: limit_price = best_ask - price_tick_size
                print(f"  Targeting BUY Limit Price at Best Ask: {limit_price}")
            elif action == 'SELL':
                # Place sell slightly ABOVE current best bid (or at it)
                limit_price = best_bid
                # More aggressive maker: limit_price = best_bid + price_tick_size
                print(f"  Targeting SELL Limit Price at Best Bid: {limit_price}")

            if limit_price <= 0:
                 print("  Could not determine valid limit price. Skipping.")
                 continue

            # --- 3. Adjust Price Precision (PRICE_FILTER tickSize) ---
            adjusted_limit_price = limit_price
            if 'PRICE_FILTER' in filters:
                tick_size = filters['PRICE_FILTER'].get('tickSize')
                if tick_size:
                    original_price = adjusted_limit_price
                    # We adjust price slightly AWAY from market for maker.
                    # If BUY, price needs to be <= best_ask. If SELL, >= best_bid.
                    # The adjust_price_to_tick rounds based on side to help.
                    adjusted_limit_price = adjust_price_to_tick(adjusted_limit_price, tick_size, action)
                    if adjusted_limit_price != original_price:
                         print(f"  Adjusted limit price for tickSize ({tick_size}): {original_price:.8f} -> {adjusted_limit_price:.8f}")
                else: print(f"  Warning: tickSize not found in PRICE_FILTER for {symbol}")
            else: print(f"  Warning: PRICE_FILTER not found for {symbol}")


            # --- 4. Adjust Quantity Precision (LOT_SIZE stepSize) ---
            adjusted_quantity = quantity
            if 'LOT_SIZE' in filters:
                step_size = filters['LOT_SIZE'].get('stepSize')
                if step_size:
                    original_qty = adjusted_quantity
                    adjusted_quantity = adjust_quantity_to_step(adjusted_quantity, step_size)
                    if adjusted_quantity != original_qty:
                         print(f"  Adjusted quantity for LOT_SIZE ({step_size}): {original_qty:.8f} -> {adjusted_quantity:.8f}")
                    if adjusted_quantity <= 0:
                         print("  Adjusted quantity is zero or less. Skipping trade.")
                         continue
                else: print(f"  Warning: stepSize not found in LOT_SIZE for {symbol}")
            else: print(f"  Warning: LOT_SIZE filter not found for {symbol}")

            # --- 5. Check Minimum Order Requirements (MIN_NOTIONAL) ---
            order_value = adjusted_quantity * adjusted_limit_price # Value at LIMIT price
            min_notional_passed = True
            if 'MIN_NOTIONAL' in filters:
                min_notional = float(filters['MIN_NOTIONAL'].get('minNotional', 0.0))
                if order_value < min_notional:
                    print(f"  Order value ({order_value:.4f}) is below MIN_NOTIONAL ({min_notional:.4f}). Skipping trade.")
                    min_notional_passed = False
                else:
                     print(f"  Order value ({order_value:.4f}) meets MIN_NOTIONAL ({min_notional:.4f}).")

            elif 'NOTIONAL' in filters: # Some symbols might use NOTIONAL instead
                 min_notional = float(filters['NOTIONAL'].get('minNotional', 0.0))
                 if order_value < min_notional:
                      print(f"  Order value ({order_value:.4f}) is below NOTIONAL minNotional ({min_notional:.4f}). Skipping trade.")
                      min_notional_passed = False
                 else:
                      print(f"  Order value ({order_value:.4f}) meets NOTIONAL minNotional ({min_notional:.4f}).")
            else:
                print(f"  Warning: No MIN_NOTIONAL/NOTIONAL filter found for {symbol}. Assuming requirement met.")


            # --- If all checks pass, format the LIMIT order ---
            if min_notional_passed and adjusted_quantity > 0 and adjusted_limit_price > 0:
                print(f"  Order for {symbol} meets requirements. Adding LIMIT order to list.")
                limit_orders_to_test.append({
                    'symbol': symbol,
                    'side': action, # 'BUY' or 'SELL'
                    'type': 'LIMIT', # Changed to LIMIT
                    'timeInForce': 'GTC', # Good 'Til Cancelled for maker orders
                    'quantity': adjusted_quantity,
                    'price': f"{adjusted_limit_price:.8f}" # Price must be sent as a string
                })

        # --- Display Formatted Orders ---
        print("\n--- Potential LIMIT Orders Ready for Testing (Maker Strategy) ---")
        if not limit_orders_to_test:
            print("No trades met the requirements after formatting for LIMIT orders.")
        else:
            limit_orders_df = pd.DataFrame(limit_orders_to_test)
            print(limit_orders_df)

except NameError as ne:
     print(f"\nError: A required variable was not found ({ne}).")
except ConnectionError as ce:
     print(f"Connection Error: {ce}") 
except Exception as e:
    print(f"\nAn unexpected error occurred during LIMIT order formatting: {e}")

Binance client initialized successfully for LIMIT order formatting.

Formatting proposed trades into LIMIT order parameters (Maker Strategy)...
Fetching exchange information...
Exchange information received.

Processing: SELL 0.00033665 BTC (BTCUSDT)
  Best Bid: 78592.64, Best Ask: 78802.14
  Targeting SELL Limit Price at Best Bid: 78592.64
  Adjusted quantity for LOT_SIZE (0.00001000): 0.00033665 -> 0.00033000
  Order value (25.9356) meets MIN_NOTIONAL (1.0000).
  Order for BTCUSDT meets requirements. Adding LIMIT order to list.

Processing: BUY 1.88052132 ADA (ADAUSDT)
  Best Bid: 0.5791, Best Ask: 0.5802
  Targeting BUY Limit Price at Best Ask: 0.5802
  Adjusted quantity for LOT_SIZE (0.10000000): 1.88052132 -> 1.80000000
  Order value (1.0444) meets MIN_NOTIONAL (1.0000).
  Order for ADAUSDT meets requirements. Adding LIMIT order to list.

Processing: BUY 4.15784685 DOGE (DOGEUSDT)
  Best Bid: 0.14828, Best Ask: 0.1487
  Targeting BUY Limit Price at Best Ask: 0.1487
  Adjusted quan

In [28]:
import pandas as pd
import os
from binance.client import Client # Make sure library is installed
from dotenv import load_dotenv     # Make sure library is installed

# --- Load API Keys and Initialize Client ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None # Initialize client as None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping() # Quick check
        print("Binance client initialized successfully for LIMIT test orders.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Assume 'limit_orders_to_test' list (or 'limit_orders_df') exists from the previous step ---
# Let's use the list 'limit_orders_to_test' if it exists
try:
    if not client:
         raise ConnectionError("Binance client not initialized. Cannot proceed.")

    orders_to_test = []
    if 'limit_orders_to_test' in locals() and limit_orders_to_test:
         orders_to_test = limit_orders_to_test
         print(f"Using 'limit_orders_to_test' list ({len(orders_to_test)} orders).")
    # Fallback to DataFrame if the list wasn't created (e.g., if cell was modified)
    elif 'limit_orders_df' in locals() and not limit_orders_df.empty:
         orders_to_test = limit_orders_df.to_dict('records')
         print(f"Using 'limit_orders_df' DataFrame ({len(orders_to_test)} orders).")
    else:
         print("Error: No formatted LIMIT orders found ('limit_orders_to_test' or 'limit_orders_df').")
         print("Please run the previous LIMIT order formatting cell first.")
         orders_to_test = None # Explicitly set to None if no orders

    if orders_to_test: # Proceed only if we have orders
        print("\n--- Sending LIMIT Test Orders (Simulation - No Real Trades) ---")

        successful_tests = 0
        failed_tests = 0

        for order_params in orders_to_test:
            print(f"Attempting TEST {order_params['side']} for {order_params['quantity']} {order_params['symbol']} at price {order_params['price']}...")
            try:
                # Send the test order - include price and timeInForce
                test_result = client.create_test_order(
                    symbol=order_params['symbol'],
                    side=order_params['side'],
                    type=order_params['type'], # Should be 'LIMIT'
                    timeInForce=order_params['timeInForce'], # Should be 'GTC'
                    quantity=order_params['quantity'],
                    price=order_params['price'] # Price is required for LIMIT orders
                )

                # A successful test order returns an empty dictionary {}
                if isinstance(test_result, dict) and not test_result:
                    print(f"  ✅ SUCCESS: Test LIMIT order for {order_params['symbol']} validated.")
                    successful_tests += 1
                else:
                    print(f"  ⚠️ UNEXPECTED RESULT (Expected empty dict): {test_result}")
                    failed_tests += 1

            except Exception as e:
                print(f"  ❌ FAILED: Test LIMIT order for {order_params['symbol']} failed validation.")
                print(f"     Error: {e}")
                failed_tests += 1

        print("\n--- LIMIT Test Order Summary ---")
        print(f"Successful Validations: {successful_tests}")
        print(f"Failed Validations    : {failed_tests}")

        if failed_tests > 0:
            print("\nReview failed test orders and adjust parameters or logic.")

except NameError as ne:
     print(f"\nError: A required variable was not found ({ne}).")
except ConnectionError as ce:
     print(f"Connection Error: {ce}")
except Exception as e:
    print(f"\nAn unexpected error occurred during LIMIT test order simulation: {e}")

Binance client initialized successfully for LIMIT test orders.
Using 'limit_orders_to_test' list (5 orders).

--- Sending LIMIT Test Orders (Simulation - No Real Trades) ---
Attempting TEST SELL for 0.00033 BTCUSDT at price 78592.64000000...
  ✅ SUCCESS: Test LIMIT order for BTCUSDT validated.
Attempting TEST BUY for 1.8 ADAUSDT at price 0.58020000...
  ✅ SUCCESS: Test LIMIT order for ADAUSDT validated.
Attempting TEST BUY for 0.003 ETHUSDT at price 1563.61000000...
  ✅ SUCCESS: Test LIMIT order for ETHUSDT validated.
Attempting TEST BUY for 0.019 SOLUSDT at price 106.96000000...
  ✅ SUCCESS: Test LIMIT order for SOLUSDT validated.
Attempting TEST BUY for 4.0 XRPUSDT at price 1.90220000...
  ✅ SUCCESS: Test LIMIT order for XRPUSDT validated.

--- LIMIT Test Order Summary ---
Successful Validations: 5
Failed Validations    : 0


In [35]:
import pandas as pd
import os
from binance.client import Client
from dotenv import load_dotenv
from decimal import Decimal, ROUND_DOWN, ROUND_UP
import json

# --- Load API Keys and Initialize Client ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping()
        print("Binance client initialized successfully for simple LIMIT test.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Configuration ---
symbol = 'ADAUSDT'
test_quantity = 1.8 # Use a quantity known to pass LOT_SIZE and MIN_NOTIONAL from previous tests

# --- Helper (reuse if needed or simplify) ---
def format_price_decimal(price, tick_size_str):
     tick_size = Decimal(tick_size_str)
     price_decimal = Decimal(str(price))
     quantizer = tick_size
     adjusted_price_decimal = price_decimal.quantize(quantizer, rounding=ROUND_DOWN)
     return adjusted_price_decimal.to_eng_string()

# --- Simple Test Logic ---
try:
    if not client:
         raise ConnectionError("Binance client not initialized.")

    print(f"\n--- Testing Simple BUY LIMIT for {symbol} ---")

    # --- Get Exchange Info & Order Book ---
    print("  Fetching exchange info & order book...")
    exchange_info = client.get_exchange_info()
    symbols_info = {item['symbol']: item for item in exchange_info['symbols']}
    if symbol not in symbols_info: raise ValueError(f"Symbol {symbol} not found.")
    symbol_info = symbols_info[symbol]
    filters = {f['filterType']: f for f in symbol_info['filters']}
    tick_size = filters.get('PRICE_FILTER', {}).get('tickSize')
    if not tick_size: raise ValueError("tickSize not found.")
    
    depth = client.get_order_book(symbol=symbol, limit=5)
    best_bid_str = depth['bids'][0][0] # Price as string from API
    best_ask_str = depth['asks'][0][0] # Price as string from API
    print(f"  Current Best Bid: {best_bid_str}, Best Ask: {best_ask_str}")

    # --- Use Best Bid as Limit Price for BUY test ---
    # (Placing it below Ask makes it a maker order initially)
    limit_price_str = format_price_decimal(best_bid_str, tick_size) # Format the bid price
    print(f"  Testing with Limit Price (Best Bid): {limit_price_str}")

    # --- Test the simple LIMIT Order ---
    print("  Sending simple BUY LIMIT test order...")
    try:
        simple_test = client.create_test_order(
            symbol=symbol,
            side='BUY',
            type='LIMIT',
            timeInForce='GTC',
            quantity=test_quantity,
            price=limit_price_str # Use formatted string
        )
        if isinstance(simple_test, dict) and not simple_test:
            print("    ✅ SUCCESS: Simple BUY LIMIT test order validated.")
        else:
            print(f"    ❌ FAILED: Simple BUY LIMIT test failed validation: {simple_test}")

    except Exception as e:
         print(f"  ❌ FAILED: Error during simple test order: {e}")


except NameError as ne:
     print(f"\nError: A required variable was not found ({ne}).")
except ConnectionError as ce:
     print(f"Connection Error: {ce}")
except ValueError as ve:
     print(f"Value Error: {ve}")
except Exception as e:
    print(f"\nAn unexpected error occurred during simple test: {e}")

Binance client initialized successfully for simple LIMIT test.

--- Testing Simple BUY LIMIT for ADAUSDT ---
  Fetching exchange info & order book...
  Current Best Bid: 0.58000000, Best Ask: 0.58110000
  Testing with Limit Price (Best Bid): 0.58000000
  Sending simple BUY LIMIT test order...
    ✅ SUCCESS: Simple BUY LIMIT test order validated.


In [37]:
import pandas as pd
import os
import numpy as np
from binance.client import Client
from dotenv import load_dotenv
from decimal import Decimal, ROUND_DOWN, ROUND_UP, getcontext
import json

# --- Load API Keys and Initialize Client ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping()
        print("Binance client initialized successfully for OCO test (CLOSER TP).") # Indicate test change
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Configuration & Hypothetical Scenario ---
asset_to_protect = 'ADA'
symbol = 'ADAUSDT'
entry_quantity = 1.8
entry_price = 0.5802

# --- Stop-Loss / Take-Profit Strategy ---
stop_loss_pct = 5.0  
take_profit_pct = 2.0 # <-- CHANGED TO 2.0% FOR TESTING ---
stop_limit_offset_pct = 0.2

# --- Helper functions ---
def format_price_decimal(price, tick_size_str):
     tick_size = Decimal(tick_size_str)
     price_decimal = Decimal(str(price))
     quantizer = tick_size
     adjusted_price_decimal = price_decimal.quantize(quantizer, rounding=ROUND_DOWN)
     return adjusted_price_decimal.to_eng_string()

def adjust_quantity_to_step(quantity, step_size_str):
    step_size = Decimal(step_size_str)
    quantity_decimal = Decimal(str(quantity))
    adjusted_quantity = (quantity_decimal // step_size) * step_size
    return float(adjusted_quantity)

# --- Main OCO Formatting Logic ---
try:
    if not client:
         raise ConnectionError("Binance client not initialized.")

    print(f"\n--- Formatting Hypothetical OCO Order for {asset_to_protect} (CLOSER TP) ---") # Indicate test change
    print(f"  Assuming entry: {entry_quantity} {asset_to_protect} at {entry_price} USDT")

    # --- Get Exchange Info for Rules ---
    print("  Fetching exchange info for OCO rules...")
    exchange_info = client.get_exchange_info()
    symbols_info = {item['symbol']: item for item in exchange_info['symbols']}

    if symbol not in symbols_info:
        raise ValueError(f"Symbol {symbol} not found in exchange info.")

    symbol_info = symbols_info[symbol]
    filters = {f['filterType']: f for f in symbol_info['filters']}
    print(f"  Rules for {symbol} obtained.")

    # --- Get Precision Rules & Price Bounds ---
    price_filter = filters.get('PRICE_FILTER', {})
    lot_size_filter = filters.get('LOT_SIZE', {})
    percent_filter = filters.get('PERCENT_PRICE', {}) 

    tick_size = price_filter.get('tickSize')
    min_price = price_filter.get('minPrice')
    max_price = price_filter.get('maxPrice')
    step_size = lot_size_filter.get('stepSize')

    if not tick_size or not step_size : # Removed check for min/max price here as they default
         raise ValueError("Essential tickSize or stepSize missing.")
         
    if not min_price: min_price = "0" # Default if missing
    if not max_price: max_price = None # Default if missing

    print(f"  Using tickSize: {tick_size}, minPrice: {min_price}, maxPrice: {max_price}")
    print(f"  Using stepSize: {step_size}")

    # --- Calculate OCO Prices ---
    take_profit_price_raw = entry_price * (1 + take_profit_pct / 100.0) # Uses 2.0% now
    stop_trigger_price_raw = entry_price * (1 - stop_loss_pct / 100.0)
    stop_limit_price_raw = stop_trigger_price_raw * (1 - stop_limit_offset_pct / 100.0)

    print(f"  Calculated TP Price (Closer): {take_profit_price_raw:.8f}") # Indicate closer TP
    print(f"  Calculated Stop Trigger: {stop_trigger_price_raw:.8f}") 
    print(f"  Calculated Stop Limit: {stop_limit_price_raw:.8f}") 

    # --- Adjust Quantity ---
    final_quantity = adjust_quantity_to_step(entry_quantity, step_size)
    print(f"  Adjusted Quantity    : {final_quantity:.8f}")

    # --- Format Prices using Decimal Quantize ---
    final_tp_price_str = format_price_decimal(take_profit_price_raw, tick_size)
    final_stop_trigger_str = format_price_decimal(stop_trigger_price_raw, tick_size)
    final_stop_limit_str = format_price_decimal(stop_limit_price_raw, tick_size)
    
    final_tp_price = float(final_tp_price_str)
    final_stop_trigger = float(final_stop_trigger_str)
    final_stop_limit = float(final_stop_limit_str)

    if final_stop_limit >= final_stop_trigger:
         print("  StopLimitPrice >= StopPrice after formatting. Adjusting StopLimit down.")
         one_tick_lower = Decimal(final_stop_limit_str) - Decimal(tick_size)
         final_stop_limit_str = format_price_decimal(one_tick_lower, tick_size)
         final_stop_limit = float(final_stop_limit_str)
         print(f"  Adjusted StopLimit downward to: {final_stop_limit_str}")
         
    print(f"  Formatted TP Price Str : {final_tp_price_str}") 
    print(f"  Formatted Stop Trigger Str: {final_stop_trigger_str}") 
    print(f"  Formatted Stop Limit Str  : {final_stop_limit_str}") 

    # --- Perform Filter Checks ---
    checks_passed = True
    f_min_price = float(min_price)
    f_max_price = float(max_price) if max_price else None 

    print("\n  Checking Price Filter Bounds...")
    print(f"    TP Price ({final_tp_price}) vs Min ({f_min_price}) / Max ({f_max_price})")
    if final_tp_price < f_min_price or (f_max_price is not None and final_tp_price > f_max_price):
        print(f"    ❌ FAILED: TP Price violates min/max bounds.")
        checks_passed = False
    else: print("    ✅ Price Filter Bounds OK.")

    if percent_filter and checks_passed:
        print("\n  Checking Percent Price Filter...")
        multiplier_up = percent_filter.get('multiplierUp') 
        multiplier_down = percent_filter.get('multiplierDown')
        if not multiplier_up or not multiplier_down:
             print("    Warning: Missing multipliers in PERCENT_PRICE filter.")
        else:
            multiplier_up = float(multiplier_up)
            multiplier_down = float(multiplier_down)
            try:
                ticker = client.get_symbol_ticker(symbol=symbol)
                current_ref_price = float(ticker['price'])
                print(f"    Reference Price (Last Ticker): {current_ref_price}")
                max_allowed_price = current_ref_price * multiplier_up
                min_allowed_price = current_ref_price * multiplier_down
                print(f"    Allowed Range (Percent Price): {min_allowed_price:.6f} - {max_allowed_price:.6f}")
                if final_tp_price > max_allowed_price or final_tp_price < min_allowed_price:
                     print(f"    ❌ FAILED: TP Price ({final_tp_price}) is outside allowed percent range.")
                     checks_passed = False
                else: print("    ✅ Percent Price OK for TP.")
            except Exception as e: print(f"    Could not perform Percent Price check: {e}")

    if checks_passed:
        print("\n  Checking MIN_NOTIONAL...")
        tp_order_value = final_quantity * final_tp_price
        min_notional_passed = True
        min_notional = 0.0
        notional_filter_key = 'MIN_NOTIONAL' if 'MIN_NOTIONAL' in filters else 'NOTIONAL' if 'NOTIONAL' in filters else None
        if notional_filter_key:
            min_notional = float(filters[notional_filter_key].get('minNotional', 0.0))
            if tp_order_value < min_notional:
                print(f"    ❌ FAILED: Take Profit order value ({tp_order_value:.4f}) is below {notional_filter_key} ({min_notional:.4f}).")
                checks_passed = False
            else: print(f"    ✅ MIN_NOTIONAL OK ({tp_order_value:.4f} >= {min_notional:.4f}).")
        else: print(f"    Warning: No MIN_NOTIONAL/NOTIONAL filter found.")
             
    # --- Prepare OCO parameters if all checks valid ---
    if checks_passed and final_quantity > 0 and final_tp_price > 0 and final_stop_trigger > 0 and final_stop_limit > 0:
        oco_params = {
            'symbol': symbol, 'side': 'SELL', 'quantity': final_quantity,
            'price': final_tp_price_str, 'stopPrice': final_stop_trigger_str,
            'stopLimitPrice': final_stop_limit_str, 'stopLimitTimeInForce': 'GTC'
        }
        print("\n--- OCO Order Parameters Formatted (Closer TP) ---")
        print(json.dumps(oco_params, indent=4)) 

        # --- Test the OCO Order Components ---
        print("\n--- Sending OCO Component Test Orders (Simulation) ---")
        try:
            print("  Testing LIMIT (Take Profit) component...")
            tp_test = client.create_test_order(symbol=symbol, side='SELL', type='LIMIT', timeInForce='GTC', quantity=final_quantity, price=oco_params['price']) 
            if isinstance(tp_test, dict) and not tp_test: print("    ✅ TP component validated.")
            else: print(f"    ❌ TP component failed validation: {tp_test}")

            print("  Testing STOP_LOSS_LIMIT component...")
            sl_test = client.create_test_order(symbol=symbol, side='SELL', type='STOP_LOSS_LIMIT', timeInForce='GTC', quantity=final_quantity, price=oco_params['stopLimitPrice'], stopPrice=oco_params['stopPrice']) 
            if isinstance(sl_test, dict) and not sl_test: print("    ✅ SL component validated.")
            else: print(f"    ❌ SL component failed validation: {sl_test}")

        except Exception as e:
             print(f"  ❌ FAILED: Error during OCO component test: {e}")

    else:
        print("\nOCO order parameters invalid or did not meet exchange filter requirements (CLOSER TP). Cannot test.") # Indicate closer TP


except NameError as ne:
     print(f"\nError: A required variable was not found ({ne}).")
except ConnectionError as ce:
     print(f"Connection Error: {ce}")
except ValueError as ve:
     print(f"Value Error: {ve}")
except Exception as e:
    print(f"\nAn unexpected error occurred during OCO formatting/testing: {e}")

Binance client initialized successfully for OCO test (CLOSER TP).

--- Formatting Hypothetical OCO Order for ADA (CLOSER TP) ---
  Assuming entry: 1.8 ADA at 0.5802 USDT
  Fetching exchange info for OCO rules...
  Rules for ADAUSDT obtained.
  Using tickSize: 0.00010000, minPrice: 0.00010000, maxPrice: 1000.00000000
  Using stepSize: 0.10000000
  Calculated TP Price (Closer): 0.59180400
  Calculated Stop Trigger: 0.55119000
  Calculated Stop Limit: 0.55008762
  Adjusted Quantity    : 1.80000000
  Formatted TP Price Str : 0.59180400
  Formatted Stop Trigger Str: 0.55119000
  Formatted Stop Limit Str  : 0.55008762

  Checking Price Filter Bounds...
    TP Price (0.591804) vs Min (0.0001) / Max (1000.0)
    ✅ Price Filter Bounds OK.

  Checking Percent Price Filter...
    Reference Price (Last Ticker): 0.5786
    Allowed Range (Percent Price): 0.115720 - 2.893000
    ✅ Percent Price OK for TP.

  Checking MIN_NOTIONAL...
    ✅ MIN_NOTIONAL OK (1.0652 >= 1.0000).

--- OCO Order Parameters 

In [38]:
import pandas as pd
import os
import numpy as np
from binance.client import Client
from dotenv import load_dotenv
from decimal import Decimal, ROUND_DOWN, ROUND_UP, getcontext
import json

# --- Load API Keys and Initialize Client ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping()
        print("Binance client initialized successfully for BTC OCO test.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Configuration & Fetching Actual BTC State ---
asset_to_protect = 'BTC'
symbol = 'BTCUSDT'
quote_asset = 'USDT'

entry_quantity = 0.0
entry_price = 0.0

# --- Function to Get Balances (Simplified) ---
def get_asset_balance(api_client, asset_symbol):
    if not api_client: return 0.0
    try:
        balance_info = api_client.get_asset_balance(asset=asset_symbol)
        return float(balance_info['free']) + float(balance_info['locked'])
    except Exception as e:
        print(f"Error getting balance for {asset_symbol}: {e}")
        return 0.0

# --- Helper functions (Price/Qty Formatting) ---
def format_price_decimal(price, tick_size_str):
     tick_size = Decimal(tick_size_str)
     price_decimal = Decimal(str(price))
     quantizer = tick_size
     adjusted_price_decimal = price_decimal.quantize(quantizer, rounding=ROUND_DOWN)
     return adjusted_price_decimal.to_eng_string()

def adjust_quantity_to_step(quantity, step_size_str):
    step_size = Decimal(step_size_str)
    quantity_decimal = Decimal(str(quantity))
    adjusted_quantity = (quantity_decimal // step_size) * step_size
    return float(adjusted_quantity)

# --- Main OCO Formatting Logic for BTC ---
try:
    if not client:
         raise ConnectionError("Binance client not initialized.")

    # --- Get Current State for BTC ---
    print(f"Fetching current state for {asset_to_protect}...")
    entry_quantity = get_asset_balance(client, asset_to_protect)
    if entry_quantity <= 0:
        raise ValueError(f"You do not hold any {asset_to_protect} to protect.")
    
    ticker = client.get_symbol_ticker(symbol=symbol)
    entry_price = float(ticker['price']) # Use current price as pseudo 'entry'
    print(f"  Current Holding: {entry_quantity} {asset_to_protect}")
    print(f"  Using current price as reference entry: {entry_price} {quote_asset}")

    print(f"\n--- Formatting Hypothetical OCO Order for {asset_to_protect} ---")

    # --- Get Exchange Info ---
    print("  Fetching exchange info for OCO rules...")
    exchange_info = client.get_exchange_info()
    symbols_info = {item['symbol']: item for item in exchange_info['symbols']}
    if symbol not in symbols_info: raise ValueError(f"Symbol {symbol} not found.")
    symbol_info = symbols_info[symbol]
    filters = {f['filterType']: f for f in symbol_info['filters']}
    print(f"  Rules for {symbol} obtained.")

    # --- Get Filters ---
    price_filter = filters.get('PRICE_FILTER', {})
    lot_size_filter = filters.get('LOT_SIZE', {})
    percent_filter = filters.get('PERCENT_PRICE', {})
    tick_size = price_filter.get('tickSize')
    step_size = lot_size_filter.get('stepSize')
    min_price_str = price_filter.get('minPrice', "0")
    max_price_str = price_filter.get('maxPrice')

    if not tick_size or not step_size:
         raise ValueError("Essential tickSize or stepSize missing.")

    print(f"  Using tickSize: {tick_size}, stepSize: {step_size}")

    # --- Stop-Loss / Take-Profit Strategy (EXAMPLES) ---
    stop_loss_pct = 5.0  
    take_profit_pct = 10.0 # Use a larger TP again for BTC
    stop_limit_offset_pct = 0.2 
    
    # --- Calculate OCO Prices ---
    take_profit_price_raw = entry_price * (1 + take_profit_pct / 100.0) 
    stop_trigger_price_raw = entry_price * (1 - stop_loss_pct / 100.0)
    stop_limit_price_raw = stop_trigger_price_raw * (1 - stop_limit_offset_pct / 100.0)

    # --- Adjust Quantity & Format Prices ---
    final_quantity = adjust_quantity_to_step(entry_quantity, step_size)
    final_tp_price_str = format_price_decimal(take_profit_price_raw, tick_size)
    final_stop_trigger_str = format_price_decimal(stop_trigger_price_raw, tick_size)
    final_stop_limit_str = format_price_decimal(stop_limit_price_raw, tick_size)
    
    final_tp_price = float(final_tp_price_str)
    final_stop_trigger = float(final_stop_trigger_str)
    final_stop_limit = float(final_stop_limit_str)

    if final_stop_limit >= final_stop_trigger:
         print("  StopLimitPrice >= StopPrice after formatting. Adjusting StopLimit down.")
         one_tick_lower = Decimal(final_stop_limit_str) - Decimal(tick_size)
         final_stop_limit_str = format_price_decimal(one_tick_lower, tick_size)
         final_stop_limit = float(final_stop_limit_str)
         print(f"  Adjusted StopLimit downward to: {final_stop_limit_str}")
         
    print(f"  Adjusted Quantity    : {final_quantity:.8f}")
    print(f"  Formatted TP Price Str : {final_tp_price_str}") 
    print(f"  Formatted Stop Trigger Str: {final_stop_trigger_str}") 
    print(f"  Formatted Stop Limit Str  : {final_stop_limit_str}") 

    # --- Perform Filter Checks ---
    checks_passed = True
    # (Add min/max/percent/min_notional checks here - simplified for brevity, 
    #  but reuse logic from Action Item 50 if needed for robustness)
    print("\n  Performing simplified filter checks...")
    # Check Quantity > 0
    if final_quantity <= 0:
        print("  ❌ FAILED: Adjusted quantity is zero or less.")
        checks_passed = False
    else: print("    ✅ Quantity OK.")
    # Check MIN_NOTIONAL (approximate)
    tp_order_value = final_quantity * final_tp_price
    min_notional = float(filters.get('MIN_NOTIONAL', {}).get('minNotional', 0.0))
    if min_notional > 0 and tp_order_value < min_notional:
         print(f"    ❌ FAILED: TP Order value ({tp_order_value:.2f}) below MIN_NOTIONAL ({min_notional:.2f}).")
         checks_passed = False
    else: print(f"    ✅ MIN_NOTIONAL OK (Value: {tp_order_value:.2f}).")


    # --- Prepare OCO parameters if valid ---
    if checks_passed and final_tp_price > 0 and final_stop_trigger > 0 and final_stop_limit > 0:
        oco_params = {
            'symbol': symbol, 'side': 'SELL', 'quantity': final_quantity,
            'price': final_tp_price_str, 'stopPrice': final_stop_trigger_str,
            'stopLimitPrice': final_stop_limit_str, 'stopLimitTimeInForce': 'GTC'
        }
        print("\n--- OCO Order Parameters Formatted for BTC---")
        print(json.dumps(oco_params, indent=4)) 

        # --- Test the OCO Order Components ---
        print("\n--- Sending OCO Component Test Orders for BTC (Simulation) ---")
        try:
            print("  Testing LIMIT (Take Profit) component...")
            tp_test = client.create_test_order(symbol=symbol, side='SELL', type='LIMIT', timeInForce='GTC', quantity=final_quantity, price=oco_params['price']) 
            if isinstance(tp_test, dict) and not tp_test: print("    ✅ TP component validated.")
            else: print(f"    ❌ TP component failed validation: {tp_test}")

            print("  Testing STOP_LOSS_LIMIT component...")
            sl_test = client.create_test_order(symbol=symbol, side='SELL', type='STOP_LOSS_LIMIT', timeInForce='GTC', quantity=final_quantity, price=oco_params['stopLimitPrice'], stopPrice=oco_params['stopPrice']) 
            if isinstance(sl_test, dict) and not sl_test: print("    ✅ SL component validated.")
            else: print(f"    ❌ SL component failed validation: {sl_test}")

        except Exception as e:
             print(f"  ❌ FAILED: Error during OCO component test: {e}")

    else:
        print("\nOCO order parameters invalid or did not meet requirements for BTC. Cannot test.")


except NameError as ne:
     print(f"\nError: A required variable was not found ({ne}).")
except ConnectionError as ce:
     print(f"Connection Error: {ce}")
except ValueError as ve:
     print(f"Value Error: {ve}")
except Exception as e:
    print(f"\nAn unexpected error occurred during BTC OCO formatting/testing: {e}")

Binance client initialized successfully for BTC OCO test.
Fetching current state for BTC...
  Current Holding: 0.0004271 BTC
  Using current price as reference entry: 78541.16 USDT

--- Formatting Hypothetical OCO Order for BTC ---
  Fetching exchange info for OCO rules...
  Rules for BTCUSDT obtained.
  Using tickSize: 0.01000000, stepSize: 0.00001000
  Adjusted Quantity    : 0.00042000
  Formatted TP Price Str : 86395.27600000
  Formatted Stop Trigger Str: 74614.10200000
  Formatted Stop Limit Str  : 74464.87379600

  Performing simplified filter checks...
    ✅ Quantity OK.
    ✅ MIN_NOTIONAL OK (Value: 36.29).

--- OCO Order Parameters Formatted for BTC---
{
    "symbol": "BTCUSDT",
    "side": "SELL",
    "quantity": 0.00042,
    "price": "86395.27600000",
    "stopPrice": "74614.10200000",
    "stopLimitPrice": "74464.87379600",
    "stopLimitTimeInForce": "GTC"
}

--- Sending OCO Component Test Orders for BTC (Simulation) ---
  Testing LIMIT (Take Profit) component...
  ❌ FAILED

In [40]:
# --- Get Current State for BTC ---
print(f"Fetching current state for {asset_to_protect}...")
entry_quantity = get_asset_balance(client, asset_to_protect)
if entry_quantity <= 0:
    raise ValueError(f"You do not hold any {asset_to_protect} to protect.")

# --- Get Mark Price as Reference ---
try:
    mark_price_info = client.get_mark_price(symbol=symbol)
    entry_price = float(mark_price_info['markPrice']) # Use Mark Price as reference
    print(f"  Using Mark Price as reference entry: {entry_price} {quote_asset}")
except Exception as e:
     print(f"Could not get Mark Price, falling back to Ticker Price. Error: {e}")
     ticker = client.get_symbol_ticker(symbol=symbol)
     entry_price = float(ticker['price'])
     print(f"  Using Ticker Price as reference entry: {entry_price} {quote_asset}")

print(f"  Current Holding: {entry_quantity} {asset_to_protect}")
print(f"\n--- Formatting Hypothetical OCO Order for {asset_to_protect} ---")

# --- Get Exchange Info ---
print("  Fetching exchange info for OCO rules...")
exchange_info = client.get_exchange_info()
symbols_info = {item['symbol']: item for item in exchange_info['symbols']}
if symbol not in symbols_info: raise ValueError(f"Symbol {symbol} not found.")
symbol_info = symbols_info[symbol]
filters = {f['filterType']: f for f in symbol_info['filters']}
print(f"  Rules for {symbol} obtained.")

# --- Get Filters ---
price_filter = filters.get('PRICE_FILTER', {})
lot_size_filter = filters.get('LOT_SIZE', {})
percent_filter = filters.get('PERCENT_PRICE', {})
tick_size = price_filter.get('tickSize')
step_size = lot_size_filter.get('stepSize')
min_price_str = price_filter.get('minPrice', "0")
max_price_str = price_filter.get('maxPrice')

if not tick_size or not step_size:
     raise ValueError("Essential tickSize or stepSize missing.")
print(f"  Using tickSize: {tick_size}, stepSize: {step_size}")

# --- Stop-Loss / Take-Profit Strategy ---
stop_loss_pct = 5.0
take_profit_pct = 10.0 # Back to 10% target
stop_limit_offset_pct = 0.2

# --- Calculate OCO Prices ---
take_profit_price_raw = entry_price * (1 + take_profit_pct / 100.0)
stop_trigger_price_raw = entry_price * (1 - stop_loss_pct / 100.0)
stop_limit_price_raw = stop_trigger_price_raw * (1 - stop_limit_offset_pct / 100.0)

print(f"  Raw Calculated TP Price : {take_profit_price_raw:.8f}")

# --- Apply PERCENT_PRICE Filter Adjustment BEFORE Formatting ---
if percent_filter:
    multiplier_up = percent_filter.get('multiplierUp')
    multiplier_down = percent_filter.get('multiplierDown')
    if multiplier_up and multiplier_down:
         multiplier_up = float(multiplier_up)
         multiplier_down = float(multiplier_down)
         max_allowed_price = entry_price * multiplier_up # Use entry (mark/ticker) price as reference
         min_allowed_price = entry_price * multiplier_down
         print(f"  Percent Price Allowed Range: {min_allowed_price:.6f} - {max_allowed_price:.6f}")
         
         # Clamp TP Price if it exceeds the max allowed by percent filter
         if take_profit_price_raw > max_allowed_price:
              print(f"  Clamping TP Price ({take_profit_price_raw:.8f}) down to Max Allowed ({max_allowed_price:.8f}) due to PERCENT_PRICE.")
              take_profit_price_raw = max_allowed_price
         # Clamp SL prices if needed (less common for TP failure)
         # if stop_trigger_price_raw < min_allowed_price: # ... clamp logic ...
         # if stop_limit_price_raw < min_allowed_price: # ... clamp logic ...
    else:
        print("  Warning: Percent price multipliers missing, cannot apply filter.")


# --- Adjust Quantity & Format Prices ---
final_quantity = adjust_quantity_to_step(entry_quantity, step_size)
final_tp_price_str = format_price_decimal(take_profit_price_raw, tick_size) # Format potentially clamped price
final_stop_trigger_str = format_price_decimal(stop_trigger_price_raw, tick_size)
final_stop_limit_str = format_price_decimal(stop_limit_price_raw, tick_size)

final_tp_price = float(final_tp_price_str)
final_stop_trigger = float(final_stop_trigger_str)
final_stop_limit = float(final_stop_limit_str)

if final_stop_limit >= final_stop_trigger:
     print("  Adjusting stopLimitPrice down by one tick...")
     one_tick_lower = Decimal(final_stop_limit_str) - Decimal(tick_size)
     final_stop_limit_str = format_price_decimal(one_tick_lower, tick_size)
     final_stop_limit = float(final_stop_limit_str)
     print(f"  Adjusted StopLimit downward to: {final_stop_limit_str}")

print(f"  Adjusted Quantity    : {final_quantity:.8f}")
print(f"  Formatted TP Price Str : {final_tp_price_str}")
print(f"  Formatted Stop Trigger Str: {final_stop_trigger_str}")
print(f"  Formatted Stop Limit Str  : {final_stop_limit_str}")

# --- Perform Filter Checks (Simplified - focus on MIN_NOTIONAL) ---
checks_passed = True
print("\n  Performing simplified filter checks...")
if final_quantity <= 0: checks_passed = False; print("  ❌ FAILED: Quantity <= 0.")
else: print("    ✅ Quantity OK.")

tp_order_value = final_quantity * final_tp_price
min_notional = float(filters.get('MIN_NOTIONAL', {}).get('minNotional', 0.0))
if min_notional > 0 and tp_order_value < min_notional:
     print(f"    ❌ FAILED: TP Value ({tp_order_value:.2f}) < MIN_NOTIONAL ({min_notional:.2f}).")
     checks_passed = False
else: print(f"    ✅ MIN_NOTIONAL OK (Value: {tp_order_value:.2f}).")


# --- Prepare OCO parameters if valid ---
if checks_passed and final_tp_price > 0 and final_stop_trigger > 0 and final_stop_limit > 0:
    oco_params = {
        'symbol': symbol, 'side': 'SELL', 'quantity': final_quantity,
        'price': final_tp_price_str, 'stopPrice': final_stop_trigger_str,
        'stopLimitPrice': final_stop_limit_str, 'stopLimitTimeInForce': 'GTC'
    }
    print("\n--- OCO Order Parameters Formatted for BTC---")
    print(json.dumps(oco_params, indent=4))

    # --- Test the OCO Order Components ---
    print("\n--- Sending OCO Component Test Orders for BTC (Simulation) ---")
    try:
        print("  Testing LIMIT (Take Profit) component...")
        tp_test = client.create_test_order(symbol=symbol, side='SELL', type='LIMIT', timeInForce='GTC', quantity=final_quantity, price=oco_params['price'])
        if isinstance(tp_test, dict) and not tp_test: print("    ✅ TP component validated.")
        else: print(f"    ❌ TP component failed validation: {tp_test}")

        print("  Testing STOP_LOSS_LIMIT component...")
        sl_test = client.create_test_order(symbol=symbol, side='SELL', type='STOP_LOSS_LIMIT', timeInForce='GTC', quantity=final_quantity, price=oco_params['stopLimitPrice'], stopPrice=oco_params['stopPrice'])
        if isinstance(sl_test, dict) and not sl_test: print("    ✅ SL component validated.")
        else: print(f"    ❌ SL component failed validation: {sl_test}")

    except Exception as e:
         print(f"  ❌ FAILED: Error during OCO component test: {e}")

else:
    print("\nOCO order parameters invalid or did not meet requirements for BTC. Cannot test.")

Fetching current state for BTC...
Could not get Mark Price, falling back to Ticker Price. Error: 'Client' object has no attribute 'get_mark_price'
  Using Ticker Price as reference entry: 78799.65 USDT
  Current Holding: 0.0004271 BTC

--- Formatting Hypothetical OCO Order for BTC ---
  Fetching exchange info for OCO rules...
  Rules for BTCUSDT obtained.
  Using tickSize: 0.01000000, stepSize: 0.00001000
  Raw Calculated TP Price : 86679.61500000
  Percent Price Allowed Range: 15759.930000 - 393998.250000
  Adjusted Quantity    : 0.00042000
  Formatted TP Price Str : 86679.61500000
  Formatted Stop Trigger Str: 74859.66750000
  Formatted Stop Limit Str  : 74709.94816500

  Performing simplified filter checks...
    ✅ Quantity OK.
    ✅ MIN_NOTIONAL OK (Value: 36.41).

--- OCO Order Parameters Formatted for BTC---
{
    "symbol": "BTCUSDT",
    "side": "SELL",
    "quantity": 0.00042,
    "price": "86679.61500000",
    "stopPrice": "74859.66750000",
    "stopLimitPrice": "74709.9481650

In [41]:
import pandas as pd
import os
import numpy as np
from binance.client import Client
from dotenv import load_dotenv
# Import Decimal and specific rounding modes
from decimal import Decimal, ROUND_DOWN, ROUND_UP, getcontext 
import json

# --- Load API Keys and Initialize Client ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping()
        print("Binance client initialized successfully for BTC OCO test (Corrected Precision).")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Configuration & Fetching Actual BTC State ---
asset_to_protect = 'BTC'
symbol = 'BTCUSDT'
quote_asset = 'USDT'

entry_quantity = 0.0
entry_price = 0.0 # Will use Ticker Price as reference

# --- Function to Get Balances (Simplified) ---
def get_asset_balance(api_client, asset_symbol):
    if not api_client: return 0.0
    try:
        balance_info = api_client.get_asset_balance(asset=asset_symbol)
        return float(balance_info['free']) + float(balance_info['locked'])
    except Exception as e:
        print(f"Error getting balance for {asset_symbol}: {e}")
        return 0.0

# --- Helper functions ---
def format_price_correctly(price, tick_size_str):
     """Formats price string EXACTLY to the tickSize precision using quantize."""
     tick_size = Decimal(tick_size_str)
     price_decimal = Decimal(str(price))
     
     # Determine the correct number of decimal places for the quantizer
     # Example: tickSize '0.01' -> quantizer = Decimal('0.01')
     # Example: tickSize '0.00001' -> quantizer = Decimal('0.00001')
     quantizer = tick_size 
     
     # Quantize - ROUND_DOWN is generally safer
     adjusted_price_decimal = price_decimal.quantize(quantizer, rounding=ROUND_DOWN)
     
     # Use to_eng_string() to ensure plain formatting without scientific notation
     return adjusted_price_decimal.to_eng_string()

def adjust_quantity_to_step(quantity, step_size_str):
    step_size = Decimal(step_size_str)
    quantity_decimal = Decimal(str(quantity))
    adjusted_quantity = (quantity_decimal // step_size) * step_size
    # Important: Return as float as API usually expects float for quantity
    return float(adjusted_quantity) 

# --- Main OCO Formatting Logic for BTC ---
try:
    if not client:
         raise ConnectionError("Binance client not initialized.")

    # --- Get Current State for BTC ---
    print(f"Fetching current state for {asset_to_protect}...")
    entry_quantity = get_asset_balance(client, asset_to_protect)
    if entry_quantity <= 0:
        raise ValueError(f"You do not hold any {asset_to_protect} to protect.")
    
    # Use Ticker Price as Reference (since get_mark_price failed)
    try:
        ticker = client.get_symbol_ticker(symbol=symbol)
        entry_price = float(ticker['price'])
        print(f"  Using Ticker Price as reference entry: {entry_price} {quote_asset}")
    except Exception as e:
        raise ValueError(f"Could not get ticker price for {symbol}: {e}")
        
    print(f"  Current Holding: {entry_quantity} {asset_to_protect}")
    print(f"\n--- Formatting Hypothetical OCO Order for {asset_to_protect} (Corrected Precision) ---")

    # --- Get Exchange Info & Filters ---
    print("  Fetching exchange info...")
    exchange_info = client.get_exchange_info()
    symbols_info = {item['symbol']: item for item in exchange_info['symbols']}
    if symbol not in symbols_info: raise ValueError(f"Symbol {symbol} not found.")
    symbol_info = symbols_info[symbol]
    filters = {f['filterType']: f for f in symbol_info['filters']}
    
    price_filter = filters.get('PRICE_FILTER', {})
    lot_size_filter = filters.get('LOT_SIZE', {})
    tick_size = price_filter.get('tickSize')
    step_size = lot_size_filter.get('stepSize')
    min_notional = float(filters.get('MIN_NOTIONAL', {}).get('minNotional', 0.0))

    if not tick_size or not step_size:
         raise ValueError("Essential tickSize or stepSize missing.")
         
    print(f"  Using tickSize: {tick_size}, stepSize: {step_size}, minNotional: {min_notional}")

    # --- Stop-Loss / Take-Profit Strategy ---
    stop_loss_pct = 5.0  
    take_profit_pct = 10.0 
    stop_limit_offset_pct = 0.2 
    
    # --- Calculate OCO Prices ---
    take_profit_price_raw = entry_price * (1 + take_profit_pct / 100.0) 
    stop_trigger_price_raw = entry_price * (1 - stop_loss_pct / 100.0)
    stop_limit_price_raw = stop_trigger_price_raw * (1 - stop_limit_offset_pct / 100.0)

    # --- Adjust Quantity & Format Prices CORRECTLY ---
    final_quantity = adjust_quantity_to_step(entry_quantity, step_size)
    
    # Use the CORRECT formatting function
    final_tp_price_str = format_price_correctly(take_profit_price_raw, tick_size)
    final_stop_trigger_str = format_price_correctly(stop_trigger_price_raw, tick_size)
    final_stop_limit_str = format_price_correctly(stop_limit_price_raw, tick_size)
    
    # Convert back for calculations/checks
    final_tp_price = float(final_tp_price_str)
    final_stop_trigger = float(final_stop_trigger_str)
    final_stop_limit = float(final_stop_limit_str)

    # Check/adjust stop limit vs stop trigger
    if final_stop_limit >= final_stop_trigger:
         print("  StopLimitPrice >= StopPrice after formatting. Adjusting StopLimit down.")
         one_tick_lower = Decimal(final_stop_limit_str) - Decimal(tick_size)
         final_stop_limit_str = format_price_correctly(one_tick_lower, tick_size)
         final_stop_limit = float(final_stop_limit_str)
         print(f"  Adjusted StopLimit downward to: {final_stop_limit_str}")
         
    print(f"  Adjusted Quantity    : {final_quantity:.8f}")
    print(f"  Formatted TP Price Str : {final_tp_price_str}") # Price sent to API
    print(f"  Formatted Stop Trigger Str: {final_stop_trigger_str}") # Price sent to API
    print(f"  Formatted Stop Limit Str  : {final_stop_limit_str}") # Price sent to API

    # --- Perform Simplified Checks ---
    checks_passed = True
    print("\n  Performing simplified filter checks...")
    if final_quantity <= 0: checks_passed = False; print("  ❌ FAILED: Quantity <= 0.")
    else: print("    ✅ Quantity OK.")
    
    tp_order_value = final_quantity * final_tp_price
    if min_notional > 0 and tp_order_value < min_notional:
         print(f"    ❌ FAILED: TP Value ({tp_order_value:.2f}) < MIN_NOTIONAL ({min_notional:.2f}).")
         checks_passed = False
    else: print(f"    ✅ MIN_NOTIONAL OK (Value: {tp_order_value:.2f}).")


    # --- Prepare OCO parameters if valid ---
    if checks_passed and final_tp_price > 0 and final_stop_trigger > 0 and final_stop_limit > 0:
        oco_params = {
            'symbol': symbol, 'side': 'SELL', 'quantity': final_quantity,
            'price': final_tp_price_str, # Use Correct String
            'stopPrice': final_stop_trigger_str, # Use Correct String
            'stopLimitPrice': final_stop_limit_str, # Use Correct String
            'stopLimitTimeInForce': 'GTC'
        }
        print("\n--- OCO Order Parameters Formatted for BTC (Corrected Precision) ---")
        print(json.dumps(oco_params, indent=4)) 

        # --- Test the OCO Order Components ---
        print("\n--- Sending OCO Component Test Orders for BTC (Simulation) ---")
        try:
            print("  Testing LIMIT (Take Profit) component...")
            tp_test = client.create_test_order(symbol=symbol, side='SELL', type='LIMIT', timeInForce='GTC', quantity=final_quantity, price=oco_params['price']) # Send correct string price
            if isinstance(tp_test, dict) and not tp_test: print("    ✅ TP component validated.")
            else: print(f"    ❌ TP component failed validation: {tp_test}")

            print("  Testing STOP_LOSS_LIMIT component...")
            sl_test = client.create_test_order(symbol=symbol, side='SELL', type='STOP_LOSS_LIMIT', timeInForce='GTC', quantity=final_quantity, price=oco_params['stopLimitPrice'], stopPrice=oco_params['stopPrice']) # Send correct string prices
            if isinstance(sl_test, dict) and not sl_test: print("    ✅ SL component validated.")
            else: print(f"    ❌ SL component failed validation: {sl_test}")

        except Exception as e:
             print(f"  ❌ FAILED: Error during OCO component test: {e}")

    else:
        print("\nOCO order parameters invalid or did not meet requirements for BTC. Cannot test.")


except NameError as ne:
     print(f"\nError: A required variable was not found ({ne}).")
except ConnectionError as ce:
     print(f"Connection Error: {ce}")
except ValueError as ve:
     print(f"Value Error: {ve}")
except Exception as e:
    print(f"\nAn unexpected error occurred during BTC OCO formatting/testing: {e}")

Binance client initialized successfully for BTC OCO test (Corrected Precision).
Fetching current state for BTC...
  Using Ticker Price as reference entry: 78864.52 USDT
  Current Holding: 0.0004271 BTC

--- Formatting Hypothetical OCO Order for BTC (Corrected Precision) ---
  Fetching exchange info...
  Using tickSize: 0.01000000, stepSize: 0.00001000, minNotional: 1.0
  Adjusted Quantity    : 0.00042000
  Formatted TP Price Str : 86750.97200000
  Formatted Stop Trigger Str: 74921.29400000
  Formatted Stop Limit Str  : 74771.45141200

  Performing simplified filter checks...
    ✅ Quantity OK.
    ✅ MIN_NOTIONAL OK (Value: 36.44).

--- OCO Order Parameters Formatted for BTC (Corrected Precision) ---
{
    "symbol": "BTCUSDT",
    "side": "SELL",
    "quantity": 0.00042,
    "price": "86750.97200000",
    "stopPrice": "74921.29400000",
    "stopLimitPrice": "74771.45141200",
    "stopLimitTimeInForce": "GTC"
}

--- Sending OCO Component Test Orders for BTC (Simulation) ---
  Testing LIM

In [42]:
import os
from binance.client import Client
from dotenv import load_dotenv
from decimal import Decimal, ROUND_DOWN, getcontext
import json

# --- Load API Keys and Initialize Client ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping()
        print("Binance client initialized successfully for Simple TP Limit Test.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Configuration ---
symbol = 'BTCUSDT'
asset = 'BTC'
take_profit_pct = 10.0 # Target percentage

# --- Helper functions ---
def format_price_correctly(price, tick_size_str):
     """Formats price string EXACTLY to the tickSize precision using quantize."""
     tick_size = Decimal(tick_size_str)
     price_decimal = Decimal(str(price))
     quantizer = tick_size
     adjusted_price_decimal = price_decimal.quantize(quantizer, rounding=ROUND_DOWN)
     return adjusted_price_decimal.to_eng_string()

def adjust_quantity_to_step(quantity, step_size_str):
    step_size = Decimal(step_size_str)
    quantity_decimal = Decimal(str(quantity))
    adjusted_quantity = (quantity_decimal // step_size) * step_size
    return float(adjusted_quantity)

def get_asset_balance(api_client, asset_symbol):
    # (Same function as before)
    if not api_client: return 0.0
    try:
        balance_info = api_client.get_asset_balance(asset=asset_symbol)
        return float(balance_info['free']) + float(balance_info['locked'])
    except Exception as e:
        print(f"Error getting balance for {asset_symbol}: {e}")
        return 0.0

# --- Simplified Test Logic ---
try:
    if not client:
         raise ConnectionError("Binance client not initialized.")

    print(f"\n--- Testing Simple LIMIT SELL for {symbol} at TP Price ---")

    # --- Get Current State & Exchange Info ---
    print("  Fetching exchange info & current state...")
    current_quantity = get_asset_balance(client, asset)
    if current_quantity <= 0: raise ValueError(f"No {asset} balance.")
    
    ticker = client.get_symbol_ticker(symbol=symbol)
    current_price = float(ticker['price'])
    
    exchange_info = client.get_exchange_info()
    symbols_info = {item['symbol']: item for item in exchange_info['symbols']}
    if symbol not in symbols_info: raise ValueError(f"Symbol {symbol} not found.")
    symbol_info = symbols_info[symbol]
    filters = {f['filterType']: f for f in symbol_info['filters']}
    
    tick_size = filters.get('PRICE_FILTER', {}).get('tickSize')
    step_size = filters.get('LOT_SIZE', {}).get('stepSize')
    min_notional = float(filters.get('MIN_NOTIONAL', {}).get('minNotional', 0.0))

    if not tick_size or not step_size: raise ValueError("tickSize or stepSize missing.")

    print(f"  Current {asset} Quantity: {current_quantity}")
    print(f"  Current Price: {current_price}")
    print(f"  Using tickSize: {tick_size}, stepSize: {step_size}, minNotional: {min_notional}")

    # --- Calculate & Format TP Price ---
    tp_price_raw = current_price * (1 + take_profit_pct / 100.0)
    tp_price_str = format_price_correctly(tp_price_raw, tick_size)
    tp_price = float(tp_price_str)
    print(f"  Calculated & Formatted TP Price: {tp_price_str}")

    # --- Adjust Quantity ---
    final_quantity = adjust_quantity_to_step(current_quantity, step_size)
    if final_quantity <= 0: raise ValueError("Adjusted quantity is zero.")
    print(f"  Adjusted Quantity to Sell: {final_quantity:.8f}")

    # --- Check Min Notional ---
    order_value = final_quantity * tp_price
    if min_notional > 0 and order_value < min_notional:
        raise ValueError(f"Order value ({order_value:.2f}) is below MIN_NOTIONAL ({min_notional:.2f}).")
    print(f"  Order Value ({order_value:.2f}) meets MIN_NOTIONAL.")


    # --- Test the Simple LIMIT Order ---
    print("\n  Sending Simple LIMIT SELL test order...")
    try:
        simple_test = client.create_test_order(
            symbol=symbol,
            side='SELL',
            type='LIMIT',
            timeInForce='GTC',
            quantity=final_quantity,
            price=tp_price_str # Use correctly formatted string price
        )
        if isinstance(simple_test, dict) and not simple_test:
            print("    ✅ SUCCESS: Simple LIMIT SELL test order validated.")
        else:
            print(f"    ❌ FAILED: Simple LIMIT SELL test failed validation: {simple_test}")

    except Exception as e:
         print(f"  ❌ FAILED: Error during simple test order: {e}")


except NameError as ne:
     print(f"\nError: A required variable was not found ({ne}).")
except ConnectionError as ce:
     print(f"Connection Error: {ce}")
except ValueError as ve:
     print(f"Value Error: {ve}")
except Exception as e:
    print(f"\nAn unexpected error occurred during simple test: {e}")

Binance client initialized successfully for Simple TP Limit Test.

--- Testing Simple LIMIT SELL for BTCUSDT at TP Price ---
  Fetching exchange info & current state...
  Current BTC Quantity: 0.0004271
  Current Price: 78923.99
  Using tickSize: 0.01000000, stepSize: 0.00001000, minNotional: 1.0
  Calculated & Formatted TP Price: 86816.38900000
  Adjusted Quantity to Sell: 0.00042000
  Order Value (36.46) meets MIN_NOTIONAL.

  Sending Simple LIMIT SELL test order...
  ❌ FAILED: Error during simple test order: APIError(code=-1013): Filter failure: PRICE_FILTER


In [43]:
import os
from binance.client import Client
from dotenv import load_dotenv
from decimal import Decimal, ROUND_DOWN, getcontext
import json

# --- Load API Keys and Initialize Client ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us')
        client.ping()
        print("Binance client initialized successfully for Clamped Simple TP Limit Test.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Configuration ---
symbol = 'BTCUSDT'
asset = 'BTC'
take_profit_pct = 10.0 # Target percentage

# --- Helper functions (Keep from previous) ---
def format_price_correctly(price, tick_size_str):
     tick_size = Decimal(tick_size_str)
     price_decimal = Decimal(str(price))
     quantizer = tick_size
     adjusted_price_decimal = price_decimal.quantize(quantizer, rounding=ROUND_DOWN)
     return adjusted_price_decimal.to_eng_string()

def adjust_quantity_to_step(quantity, step_size_str):
    step_size = Decimal(step_size_str)
    quantity_decimal = Decimal(str(quantity))
    adjusted_quantity = (quantity_decimal // step_size) * step_size
    return float(adjusted_quantity)

def get_asset_balance(api_client, asset_symbol):
    if not api_client: return 0.0
    try:
        balance_info = api_client.get_asset_balance(asset=asset_symbol)
        return float(balance_info['free']) + float(balance_info['locked'])
    except Exception as e: print(f"Error getting balance for {asset_symbol}: {e}"); return 0.0

# --- Simplified Test Logic with Clamping ---
try:
    if not client: raise ConnectionError("Binance client not initialized.")

    print(f"\n--- Testing Clamped Simple LIMIT SELL for {symbol} ---")

    # --- Get Current State & Exchange Info ---
    print("  Fetching exchange info & current state...")
    current_quantity = get_asset_balance(client, asset)
    if current_quantity <= 0: raise ValueError(f"No {asset} balance.")
    
    ticker = client.get_symbol_ticker(symbol=symbol)
    current_price = float(ticker['price'])
    
    exchange_info = client.get_exchange_info()
    symbols_info = {item['symbol']: item for item in exchange_info['symbols']}
    if symbol not in symbols_info: raise ValueError(f"Symbol {symbol} not found.")
    symbol_info = symbols_info[symbol]
    filters = {f['filterType']: f for f in symbol_info['filters']}
    
    price_filter = filters.get('PRICE_FILTER', {})
    lot_size_filter = filters.get('LOT_SIZE', {})
    percent_filter = filters.get('PERCENT_PRICE', {})
    tick_size = price_filter.get('tickSize')
    step_size = lot_size_filter.get('stepSize')
    min_notional = float(filters.get('MIN_NOTIONAL', {}).get('minNotional', 0.0))

    if not tick_size or not step_size: raise ValueError("tickSize or stepSize missing.")
    print(f"  Current Price: {current_price}")
    print(f"  Using tickSize: {tick_size}, stepSize: {step_size}, minNotional: {min_notional}")

    # --- Calculate TP Price ---
    tp_price_raw = current_price * (1 + take_profit_pct / 100.0)
    print(f"  Raw Calculated TP Price: {tp_price_raw:.8f}")

    # --- Clamp to PERCENT_PRICE Bounds ---
    final_tp_price_to_format = tp_price_raw # Start with raw price
    if percent_filter:
        multiplier_up = percent_filter.get('multiplierUp')
        multiplier_down = percent_filter.get('multiplierDown')
        if multiplier_up and multiplier_down:
             multiplier_up = float(multiplier_up)
             multiplier_down = float(multiplier_down)
             # Use current_price as the reference for bounds calc
             max_allowed_price = current_price * multiplier_up
             min_allowed_price = current_price * multiplier_down
             print(f"  Percent Price Allowed Range: [{min_allowed_price:.6f} - {max_allowed_price:.6f}]")
             
             # CLAMP the raw price before formatting
             if tp_price_raw > max_allowed_price:
                  print(f"  !!! Clamping TP Price ({tp_price_raw:.8f}) down to Max Allowed ({max_allowed_price:.8f}) !!!")
                  final_tp_price_to_format = max_allowed_price
             elif tp_price_raw < min_allowed_price: # Also check lower bound just in case
                   print(f"  !!! Clamping TP Price ({tp_price_raw:.8f}) up to Min Allowed ({min_allowed_price:.8f}) !!!")
                   final_tp_price_to_format = min_allowed_price
             else:
                  print("  TP Price is within Percent Price bounds.")
        else: print("  Warning: Percent price multipliers missing.")

    # --- Format Price and Adjust Quantity ---
    tp_price_str = format_price_correctly(final_tp_price_to_format, tick_size)
    tp_price = float(tp_price_str)
    print(f"  Price after Clamping & Formatting: {tp_price_str}")

    final_quantity = adjust_quantity_to_step(current_quantity, step_size)
    if final_quantity <= 0: raise ValueError("Adjusted quantity is zero.")
    print(f"  Adjusted Quantity to Sell: {final_quantity:.8f}")

    # --- Check Min Notional ---
    order_value = final_quantity * tp_price
    if min_notional > 0 and order_value < min_notional:
        raise ValueError(f"Order value ({order_value:.2f}) is below MIN_NOTIONAL ({min_notional:.2f}).")
    print(f"  Order Value ({order_value:.2f}) meets MIN_NOTIONAL.")


    # --- Test the Simple LIMIT Order ---
    print("\n  Sending Clamped Simple LIMIT SELL test order...")
    try:
        simple_test = client.create_test_order(
            symbol=symbol, side='SELL', type='LIMIT', timeInForce='GTC',
            quantity=final_quantity, price=tp_price_str 
        )
        if isinstance(simple_test, dict) and not simple_test:
            print("    ✅ SUCCESS: Clamped LIMIT SELL test order validated.")
        else:
            print(f"    ❌ FAILED: Clamped LIMIT SELL test failed validation: {simple_test}")

    except Exception as e:
         print(f"  ❌ FAILED: Error during simple test order: {e}")


except NameError as ne: print(f"\nError: Variable not found ({ne}).")
except ConnectionError as ce: print(f"Connection Error: {ce}")
except ValueError as ve: print(f"Value Error: {ve}")
except Exception as e: print(f"\nUnexpected error: {e}")

Binance client initialized successfully for Clamped Simple TP Limit Test.

--- Testing Clamped Simple LIMIT SELL for BTCUSDT ---
  Fetching exchange info & current state...
  Current Price: 78963.97
  Using tickSize: 0.01000000, stepSize: 0.00001000, minNotional: 1.0
  Raw Calculated TP Price: 86860.36700000
  Percent Price Allowed Range: [15792.794000 - 394819.850000]
  TP Price is within Percent Price bounds.
  Price after Clamping & Formatting: 86860.36700000
  Adjusted Quantity to Sell: 0.00042000
  Order Value (36.48) meets MIN_NOTIONAL.

  Sending Clamped Simple LIMIT SELL test order...
  ❌ FAILED: Error during simple test order: APIError(code=-1013): Filter failure: PRICE_FILTER


In [44]:
import os
from binance.client import Client
from dotenv import load_dotenv
# Import Decimal and rounding modes
from decimal import Decimal, ROUND_DOWN, getcontext 
import json
import math # For logarithm calculation

# --- Load API Keys and Initialize Client ---
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

client = None
if api_key and api_secret:
    try:
        client = Client(api_key, api_secret, tld='us') # Ensure tld='us'
        client.ping()
        print("Binance client initialized successfully for FINAL Simple TP Limit Test.")
    except Exception as e:
        print(f"Error initializing Binance client: {e}")
        client = None
else:
    print("API Key or Secret not found. Cannot initialize client.")

# --- Configuration ---
symbol = 'BTCUSDT' # Spot market symbol on Binance.US
asset = 'BTC'
take_profit_pct = 10.0 # Target percentage

# --- Helper functions ---
def format_price_ticksize(price, tick_size_str):
     """Formats price string EXACTLY to the tickSize precision using Decimal quantize."""
     tick_size = Decimal(tick_size_str)
     price_decimal = Decimal(str(price))
     
     # Calculate the number of decimal places required by tick_size
     # Example: 0.01 -> -2, 0.001 -> -3
     # Use log10, handle potential edge cases if tick_size is 1 or more
     if tick_size >= 1:
          decimal_places = 0
     else:
          decimal_places = abs(math.log10(tick_size)) # abs() needed? log10(0.01) is -2
          # Check if it's an integer number of places
          if decimal_places != int(decimal_places):
               # Fallback or error for unusual tick sizes like 0.005
               print(f"Warning: Unusual tick_size {tick_size}, using default rounding.")
               # Let's just use the exponent method for safety
               decimal_places = abs(tick_size.as_tuple().exponent)
          else:
               decimal_places = int(decimal_places)

     # Create the quantizer exponent: e.g., '0.01' for 2 decimal places
     # Use string formatting to ensure correct precision representation
     quantizer_str = f"1e-{decimal_places}" 
     # Or simply use the tick_size itself if it's a power of 10
     # Let's stick to the explicit exponent method: Decimal('0.01') for 2 places
     quantizer = Decimal('1e-' + str(decimal_places))

     # Quantize using ROUND_DOWN (generally safer for filters)
     adjusted_price_decimal = price_decimal.quantize(quantizer, rounding=ROUND_DOWN)
     
     # Return as plain string using format matching the calculated decimal places
     # Use f-string formatting to guarantee the number of decimal places
     return f"{adjusted_price_decimal:.{decimal_places}f}"

def adjust_quantity_to_step(quantity, step_size_str):
    # (Same function as before)
    step_size = Decimal(step_size_str)
    quantity_decimal = Decimal(str(quantity))
    adjusted_quantity = (quantity_decimal // step_size) * step_size
    return float(adjusted_quantity)

def get_asset_balance(api_client, asset_symbol):
    # (Same function as before)
    if not api_client: return 0.0
    try:
        balance_info = api_client.get_asset_balance(asset=asset_symbol)
        return float(balance_info['free']) + float(balance_info['locked'])
    except Exception as e: print(f"Error getting balance for {asset_symbol}: {e}"); return 0.0

# --- Simplified Test Logic with CORRECT Formatting ---
try:
    if not client: raise ConnectionError("Binance client not initialized.")

    print(f"\n--- Testing FINAL Simple LIMIT SELL for {symbol} ---")

    # --- Get Current State & Exchange Info ---
    print("  Fetching exchange info & current state...")
    current_quantity = get_asset_balance(client, asset)
    if current_quantity <= 0: raise ValueError(f"No {asset} balance.")
    
    ticker = client.get_symbol_ticker(symbol=symbol)
    current_price = float(ticker['price'])
    
    exchange_info = client.get_exchange_info()
    symbols_info = {item['symbol']: item for item in exchange_info['symbols']}
    if symbol not in symbols_info: raise ValueError(f"Symbol {symbol} not found.")
    symbol_info = symbols_info[symbol]
    filters = {f['filterType']: f for f in symbol_info['filters']}
    
    price_filter = filters.get('PRICE_FILTER', {})
    lot_size_filter = filters.get('LOT_SIZE', {})
    tick_size = price_filter.get('tickSize')
    step_size = lot_size_filter.get('stepSize')
    min_notional = float(filters.get('MIN_NOTIONAL', {}).get('minNotional', 0.0))

    if not tick_size or not step_size: raise ValueError("tickSize or stepSize missing.")
    print(f"  Current Price: {current_price}")
    print(f"  Using tickSize: {tick_size}, stepSize: {step_size}, minNotional: {min_notional}")

    # --- Calculate TP Price ---
    tp_price_raw = current_price * (1 + take_profit_pct / 100.0)
    print(f"  Raw Calculated TP Price: {tp_price_raw:.8f}")

    # --- Format Price CORRECTLY ---
    tp_price_str = format_price_ticksize(tp_price_raw, tick_size) # Use the corrected function
    tp_price = float(tp_price_str)
    print(f"  CORRECTLY Formatted TP Price String: '{tp_price_str}'") # Show the exact string

    # --- Adjust Quantity ---
    final_quantity = adjust_quantity_to_step(current_quantity, step_size)
    if final_quantity <= 0: raise ValueError("Adjusted quantity is zero.")
    print(f"  Adjusted Quantity to Sell: {final_quantity:.8f}")

    # --- Check Min Notional ---
    order_value = final_quantity * tp_price
    if min_notional > 0 and order_value < min_notional:
        raise ValueError(f"Order value ({order_value:.2f}) is below MIN_NOTIONAL ({min_notional:.2f}).")
    print(f"  Order Value ({order_value:.2f}) meets MIN_NOTIONAL.")


    # --- Test the Simple LIMIT Order ---
    print("\n  Sending FINAL Simple LIMIT SELL test order...")
    try:
        simple_test = client.create_test_order(
            symbol=symbol,
            side='SELL',
            type='LIMIT',
            timeInForce='GTC',
            quantity=final_quantity,
            price=tp_price_str # Use the CORRECTLY formatted string price
        )
        if isinstance(simple_test, dict) and not simple_test:
            print("    ✅ SUCCESS: Simple LIMIT SELL test order validated.")
        else:
            print(f"    ❌ FAILED: Simple LIMIT SELL test failed validation: {simple_test}")

    except Exception as e:
         print(f"  ❌ FAILED: Error during simple test order: {e}")


except NameError as ne: print(f"\nError: Variable not found ({ne}).")
except ConnectionError as ce: print(f"Connection Error: {ce}")
except ValueError as ve: print(f"Value Error: {ve}")
except Exception as e: print(f"\nUnexpected error: {e}")

Binance client initialized successfully for FINAL Simple TP Limit Test.

--- Testing FINAL Simple LIMIT SELL for BTCUSDT ---
  Fetching exchange info & current state...
  Current Price: 78963.56
  Using tickSize: 0.01000000, stepSize: 0.00001000, minNotional: 1.0
  Raw Calculated TP Price: 86859.91600000
  CORRECTLY Formatted TP Price String: '86859.91'
  Adjusted Quantity to Sell: 0.00042000
  Order Value (36.48) meets MIN_NOTIONAL.

  Sending FINAL Simple LIMIT SELL test order...
    ✅ SUCCESS: Simple LIMIT SELL test order validated.
